# This is a jupyter to investigate how does the distributed dashboard works and also find which places in the analyzeNtuples takes most of the computation time

# 1. Import libraries

In [3]:
import numpy as np
import fsspec
import distributed
import awkward as ak
import dask_awkward as dak
import hist

import dask_histogram as dh
import boost_histogram as bh
import sys
sys.path.insert(0, '..')

sys.path.insert(1, '../python/')
print("Loaded")

Loaded


In [4]:
from dask.distributed import LocalCluster
cluster = LocalCluster()          # Fully-featured local Dask cluster
client = cluster.get_client()

/data/gsvedas/.virtualenvs/gintautas_fast/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37581 instead
  warnings.warn(


In [5]:
client.dashboard_link

'http://127.0.0.1:8787/status'

In [6]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 48,Total memory: 62.29 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:41545,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 48
Started: Just now,Total memory: 62.29 GiB
Comm: tcp://127.0.0.1:42385,Total threads: 6
Dashboard: http://127.0.0.1:33911/status,Memory: 7.79 GiB
Nanny: tcp://127.0.0.1:45143,


# 2. Example of how the params look in the analyzeNtuples

In [ ]:
params = {
    'input_base_dir':  '/eos/cms/store/cmst3/group/l1tr/cerminar/l1teg/fp_ntuples/',
    'input_sample_dir':  'DoubleElectron_FlatPt-1To100_PU200/FP/v102D/',
    'tree_name':  'Events',
    'output_filename_base':  'histos_doubleele_flat1to100_PU200_egplots_v160A.102E',
    'output_filename':  '/afs/cern.ch/user/g/gsvedas/gintautas_fast/ntuple-analysis/plots/histos_doubleele_flat1to100_PU200_egplots_v160A.102Ei.root',
    'output_dir':  '/afs/cern.ch/user/g/gsvedas/gintautas_fast/ntuple-analysis/plots/',
    'clusterize':  False,
    'eventsToDump':  [],
    'version':  'v160A.102E',
    'calib_version':  'calib-v134C',
    'rate_pt_wps':  'data/rate_pt_wps_v152B.90A.json',
    'maxEvents':  1,
    'events_per_job':  -1,
    'computeDensity':  False   
}

# 3. analyze.py

## 3.1 Imports

In [5]:
import os
import sys
import traceback

import uproot as up
from rich import print as pprint
import python.calibrations as calibs
import python.file_manager as fm
import python.tree_reader as treereader
from python import collections, timecounter

# @profile
analyze_counter = 1

import threading
import dask

Welcome to JupyROOT 6.26/11


/data/gsvedas/.virtualenvs/gintautas_fast/lib/python3.9/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/data/gsvedas/.virtualenvs/gintautas_fast/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/data/gsvedas/.virtualenvs/gintautas_fast/lib/python3.9/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/data/gsvedas/.virtualenvs/gintautas_fast/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


## 3.2 actual function

In [39]:
def analyze(params, a, batch_idx=-1):
    print("[0] [JUPYTER] ANALYZE IS ACTIVE")
     
    params.print()
    debug = int(params.debug)
    input_files = ['root://eoscms.cern.ch//eos/cms/store/cmst3/group/l1tr/cerminar/l1teg/fp_ntuples/DoubleElectron_FlatPt-1To100_PU200/FP/v102D/perfNano_9026841_0.root']
    range_ev = (0, params.maxEvents)

    pprint(f'\n- will read {len(input_files)} files from dir {params.input_sample_dir}:')
    for file_name in input_files:
        pprint(f'        - {file_name}')
    pprint('')

    files_with_protocol = [fm.get_eos_protocol(file_name) + file_name for file_name in input_files]
    print("[1] [JUPYTER] FILES WITH PROTOCOL READ")
    
    
    calib_manager = calibs.CalibManager()
    calib_manager.set_calibration_version(params.calib_version)
    if params.rate_pt_wps:
        calib_manager.set_pt_wps_version(params.rate_pt_wps)

    print("[2] [JUPYTER] CALIBRATION is done")
        
    output = up.recreate(params.output_filename)
    hm = HistoManager()
    hm.file = output
    
    print("[JUPYTER] params.plotters", params.plotters)
    
    # instantiate all the plotters
    plotter_collection = []
    plotter_collection.extend(params.plotters)

    print("[3] [JUPYTER] BEFORE BOOK HISTOS")
    # -------------------------BOOK HISTOS------------------------------

    for plotter in plotter_collection:
        plotter.book_histos()

    collection_manager = collections.EventManager()

    if params.weight_file is not None:
        collection_manager.read_weight_file(params.weight_file)
    
    print("[4] [JUPYTER] BOOK HISTOS FINISHED")
    # -------------------------EVENT LOOP--------------------------------
    print("[5] [JUPYTER] EVENT LOOP IS STARTING")
    
    tree_reader = TreeReader(range_ev, params.maxEvents)
    
    results = []
    for filename in files_with_protocol:
        #results.append(process_file_simple(filename))
        results.append(process_file(filename, tree_reader, debug, collection_manager, plotter_collection, hm, 1))
        # filename, tree_reader, params, debug, collection_manager, plotter_collection)

    pprint(f'Writing histos to file {params.output_filename}')
       
    print("[JUPYTER] WRITE HISTOS")
    dask.compute(results)
    
    hm.writeHistos()    
    output.close()

    return tree_reader.n_tot_entries

## 3.3 Experiment with dask.delayed objects

In [12]:
@dask.delayed
def process_file_simple(filename):
    tree_file = up.open(filename, num_workers=1)
    pprint(f'opening file: {filename}')
    return tree_file

In [36]:
@dask.delayed
def process_file(filename, tree_reader, debug, collection_manager, plotter_collection, hm, b):
    print("ACTIVE PROCESS_FILE FROM JUPYTERINO")
    
    tree_file = up.open(filename, num_workers=1)
    pprint(f'opening file: {filename}')

    ttree = tree_file['Events']
    tree_reader.setTree(ttree)

    while tree_reader.next(debug):
        try:
            collection_manager.read(tree_reader, debug)

            for plotter in plotter_collection:
                plotter.fill_histos_event(tree_reader.file_entry, debug=debug)

        except Exception as inst:
            pprint(f'[EXCEPTION OCCURRED:] {inst!s}')
            pprint('Unexpected error:', sys.exc_info()[0])

    tree_file.close()
        
    return None

# 4. Run analyzeNtuples

## 4.1 run the script via CLI

In [50]:
%run  ../analyzeNtuples.py -f ../cfg/egplots.yaml -i ../cfg/datasets/ntpfp_v102.yaml -c tkele_plots -s doubleele_flat1to100_PU200 -n -1 -d 0

--- Collection: tkele_plots with samples: dict_keys(['doubleele_flat1to100_PU200', 'doublephoton_flat1to100_PU200',
'nugun_alleta_pu200', 'ttbar_PU200'])

--- Collection: quantization_plots with samples: dict_keys(['doubleele_flat1to100_PU200', 
'doublephoton_flat1to100_PU200', 'nugun_alleta_pu200', 'ttbar_PU200'])

About to process samples:

========================== #1/1: doubleele_flat1to100_PU200 ==========================

SAMPLE 
--------------------Parameters--------------------
Name            : doubleele_flat1to100_PU200
clusterize      : 0           
compute density : 0           
maxEvents       : -1          
output file     : /afs/cern.ch/user/g/gsvedas/gintautas_fast/ntuple-analysis/plots/histos_doubleele_flat1to100_PU200_egplots_v160A.102Ei.root
events per job  : -1          
debug           : 0           



...

- 
/eos/cms/store/cmst3/group/l1tr/cerminar/l1teg/fp_ntuples/DoubleElectron_FlatPt-1To100_PU200/FP/v102D/perfNano_9026
841_3.root

- 
/eos/cms/store/cmst3/group/l1tr/cerminar/l1teg/fp_ntuples/DoubleElectron_FlatPt-1To100_PU200/FP/v102D/perfNano_9026
841_4.root

[CalibManager] pt wps json file version: data/rate_pt_wps_v152B.90A.json!
ABOUT TO COMPUTE...
No module named 'python.tree_reader'
Unexpected error: <class 'ModuleNotFoundError'>


Traceback (most recent call last):
  File "/afs/cern.ch/user/g/gsvedas/gintautas_fast/ntuple-analysis/plot-drawing-tools/../python/timecounter.py", line 87, in wrapper
    nevents += func(*args, **kwargs)
  File "/afs/cern.ch/user/g/gsvedas/gintautas_fast/ntuple-analysis/analyzeNtuples.py", line 124, in analyzeNtuples
    ret_nevents += analyze(sample, batch_idx=batch_idx)
  File "/afs/cern.ch/user/g/gsvedas/gintautas_fast/ntuple-analysis/plot-drawing-tools/../python/analyzer.py", line 100, in analyze
    dask.compute(results)
  File "/data/gsvedas/.virtualenvs/gintautas_fast/lib/python3.9/site-packages/dask/base.py", line 661, in compute
    results = schedule(dsk, keys, **kwargs)
  File "/cvmfs/cms.cern.ch/el8_amd64_gcc11/external/python3/3.9.14-76a14295dd5255228210eb596893b98c/lib/python3.9/contextlib.py", line 79, in inner
    return func(*args, **kwds)
  File "/data/gsvedas/.virtualenvs/gintautas_fast/lib/python3.9/site-packages/distributed/protocol/pickle.py", line 96, in loads
 

AssertionError: 

## 4.2 run the analyzeNtuples declared in this jupyter notebook 

In [ ]:
analyzeNtuples(
    configfile="../cfg/egplots.yaml",
    datasetfile="../cfg/datasets/ntpfp_v102.yaml",
    collection="tkele_plots",
    sample="doubleele_flat1to100_PU200",
    nevents=-1,
    debug=0,
)

In [46]:
%pwd

'/afs/cern.ch/user/g/gsvedas/gintautas_fast/ntuple-analysis/plot-drawing-tools'

# 5. analyzeNtuples.py

## 5.1 Imports

In [20]:
import sys

import typer
import yaml
from rich import print as pprint

from cfg import *  #!FIXXX
from python.analyzer import analyze
from python.parameters import Parameters, get_collection_parameters
from python.submission import to_HTCondor
from python.timecounter import print_stats

n: Pt2, 
	 l:p_{T}^{L1} #geq 2GeV
n: Pt5, 
	 l:p_{T}^{L1} #geq 5GeV
n: all, 
	 l:
n: Pt1, 
	 l:p_{T}^{L1}#geq1GeV
n: IDPuVetoPt2, 
	 l:PU Veto, p_{T}^{L1} #geq 2GeV
n: IDPuVetoPt5, 
	 l:PU Veto, p_{T}^{L1} #geq 5GeV
n: IDPuVeto, 
	 l:PU Veto
n: IDPuVetoPt1, 
	 l:PU Veto, p_{T}^{L1}#geq1GeV
n: IDEmPfPt2, 
	 l:PF Em, p_{T}^{L1} #geq 2GeV
n: IDEmPfPt5, 
	 l:PF Em, p_{T}^{L1} #geq 5GeV
n: IDEmPf, 
	 l:PF Em
n: IDEmPfPt1, 
	 l:PF Em, p_{T}^{L1}#geq1GeV
n: IDEmPfIDPuVetoPt2, 
	 l:PF Em, PU Veto, p_{T}^{L1} #geq 2GeV
n: IDEmPfIDPuVetoPt5, 
	 l:PF Em, PU Veto, p_{T}^{L1} #geq 5GeV
n: IDEmPfIDPuVeto, 
	 l:PF Em, PU Veto
n: IDEmPfIDPuVetoPt1, 
	 l:PF Em, PU Veto, p_{T}^{L1}#geq1GeV
n: IDEmTightPt2, 
	 l:EgID Tight, p_{T}^{L1} #geq 2GeV
n: IDEmTightPt5, 
	 l:EgID Tight, p_{T}^{L1} #geq 5GeV
n: IDEmTight, 
	 l:EgID Tight
n: IDEmTightPt1, 
	 l:EgID Tight, p_{T}^{L1}#geq1GeV
n: IDEmTightIDPuVetoPt2, 
	 l:EgID Tight, PU Veto, p_{T}^{L1} #geq 2GeV
n: IDEmTightIDPuVetoPt5, 
	 l:EgID Tight, PU Veto, p_{

## 5.2 actual function

In [28]:
def analyzeNtuples(
    configfile: str,
    datasetfile: str,
    collection: str,
    sample: str,
    debug: int = 0,
    nevents: int = 10,
    batch: int = None,
    run: str = None,
    outdir: str = None,
    local: bool = False,
    workers: int = 2,
    workdir: str = None,
    submit: bool = False,
):
    
    print("ACTIVATING THIS JUPYTER")
    
    if submit and local and not workdir:
        raise ValueError('The --workdir option is required when submitting jobs locally')

    def parse_yaml(filename):
        with open(filename) as stream:
            return yaml.load(stream, Loader=yaml.FullLoader)

    cfgfile = {}
    cfgfile.update(parse_yaml(configfile))
    cfgfile.update(parse_yaml(datasetfile))

    opt = Parameters(
        {
            'COLLECTION': collection,
            'SAMPLE': sample,
            'DEBUG': debug,
            'NEVENTS': nevents,
            'BATCH': batch,
            'RUN': run,
            'OUTDIR': outdir,
            'LOCAL': local,
            'WORKERS': workers,
            'WORKDIR': workdir,
            'SUBMIT': submit,
        }
    )
    collection_params = get_collection_parameters(opt, cfgfile)

    samples_to_process = []

    if not opt.COLLECTION:
        print(f'\nAvailable collections: {collection_params.keys()}')
        sys.exit(0)
    if opt.COLLECTION not in collection_params:
        print(f'ERROR: collection {opt.COLLECTION} not in the cfg file')
        sys.exit(10)
    if not opt.SAMPLE:
        print(f'Collection: {opt.COLLECTION}, available samples: {collection_params[opt.COLLECTION]}')
        sys.exit(0)

    if opt.SAMPLE == 'all':
        samples_to_process.extend(collection_params[opt.COLLECTION])
    else:
        sel_sample = [sample for sample in collection_params[opt.COLLECTION] if sample.name == opt.SAMPLE]
        samples_to_process.append(sel_sample[0])

    pprint(f'About to process samples: {samples_to_process}')

    plot_version = f"{cfgfile['common']['plot_version']}.{cfgfile['dataset']['version']}"

    to_HTCondor(
        analyze=analyze,
        opt=opt,
        submit_mode=submit,
        plot_version=plot_version,
        samples_to_process=samples_to_process,
    )

    batch_idx = -1
    if opt.BATCH and opt.RUN:
        batch_idx = int(opt.RUN)

    ret_nevents = 0
    
    print("SAMPLE", samples_to_process)
    
    
    for idx, sample in enumerate(samples_to_process):
        pprint(
            f'\n\n========================== #{idx+1}/{len(samples_to_process)}: {sample.name} ==========================\n'
        )
        print("SAMPLE", sample)
        ret_nevents += analyze(sample, 1000, batch_idx=batch_idx)
    print("RET EVENTS", ret_nevents)
    return ret_nevents

# 6. tree_reader.py

In [18]:
import datetime
import gc
import resource

import awkward as ak
import vector

import coffea
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema, BaseSchema

vector.register_awkward()

class TreeReader:
    def __init__(self, entry_range, max_events):
        self.tree = None
        self._branches = []
        # this is the gloabl "entry" across files
        self.global_entry = -1
        # this is the "entry" local to the open file (reset to 0) every new file
        self.file_entry = -1
        self.max_events = max_events
        self.entry_range = entry_range

        self.n_tot_entries = 0

    def setTree(self, uptree):
        self.tree = uptree
        self._branches = []
        branch_blacklist = ['tc_wafer',
                            'tc_cell',
                            'tc_waferu',
                            'tc_waferv',
                            'tc_cellu',
                            'tc_cellv',
                            'gen_PUNumInt',
                            'gen_TrueNumInt',
                            # 'gen_daughters',
                            'simpart_posx', 'simpart_posy', 'simpart_posz',
                            ]
        if len(self._branches) == 0:
            self._branches = [br for br in self.tree.keys() if br not in branch_blacklist]
        print(f'open new tree file with # entries: {self.tree.num_entries}')
        self.file_entry = -1

    def next(self, debug=0):

        if self.max_events != -1:
            if self.n_tot_entries == self.max_events:
                print('END loop for max_event!')
                # we processed the max # of events
                return False
        if self.entry_range[1] != -1:
            if self.global_entry == self.entry_range[1]:
                print('END loop for entry_range')
                return False
        if self.file_entry == self.tree.num_entries-1:
            print('END loop for end_of_file')
            return False

        if self.global_entry == -1:
            self.global_entry = self.entry_range[0]
            self.file_entry = self.entry_range[0]
        else:
            self.file_entry += 1
            self.global_entry += 1

        # entry is the cursor in the file: when we open a new one (not the first) needs to be set to 0 again
        if debug >= 2 or self.global_entry % 1000 == 0:
            self.printEntry()

        self.n_tot_entries += 1
        return True

    def printEntry(self):
        print(f'--- File entry: {self.file_entry}, global entry: {self.global_entry}, tot # events: {self.n_tot_entries} @ {datetime.datetime.now()}, MaxRSS {resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1000000.0:.2f} Mb')
        # print(self.tree.keys())
        # print(self.tree.arrays(['run', 'lumi', 'event'], library='pd', entry_start=self.file_entry, entry_stop=self.file_entry+1))
        # self.dump_garbage()

    def dump_garbage(self):
        """
        show us what's the garbage about
        """
        # force collection
        print ('\nGARBAGE:')
        gc.collect()

        print ('\nGARBAGE OBJECTS:')
        for x in gc.garbage:
            s = str(x)
            if len(s) > 80: s = s[:80]
            print (type(x),'\n  ', s)


    def getDataFrame(self, prefix, entry_block, fallback=None):
        branches = [br for br in self._branches
                    if br.startswith(f'{prefix}_') and
                    br != f'{prefix}_n']
        names = ['_'.join(br.split('_')[1:]) for br in branches]
        name_map = dict(zip(names, branches))
        if len(branches) == 0:
            if fallback is not None:
                return self.getDataFrame(prefix=fallback, entry_block=entry_block)
            prefs = set([br.split('_')[0] for br in self._branches])
            print(f'stored branch prefixes are: {prefs}')
            raise ValueError(f'[TreeReader::getDataFrame] No branches with prefix: {prefix}')

        dask_akarray = NanoEventsFactory.from_root(
                                   self.tree,
                                   schemaclass=NanoAODSchema).events()

        print("DATA FRAME FROM JUPYTER")
        #akarray = self.tree.arrays(names, 
        #library='ak', 
        #aliases=name_map, 
        #entry_start=self.file_entry, 
        #entry_stop=self.file_entry+entry_block)
        #print("[0] prefix to select: ", prefix)

        print("ACTIVATING TREE READER JUPYTER... ")

        dask_akarray = dask_akarray[prefix]     
        #print("[1] Selected fields from prefix", dask_akarray.fields)   

        dask_akarray = dask_akarray[names]

        #print("[2] specific fields with names", dask_akarray.fields)
        dask_akarray = dask_akarray[self.file_entry : self.file_entry + entry_block]

        # print(akarray)
        records = {}
        for field in dask_akarray.fields:
            records[field] = dask_akarray[field]

        if 'pt' in names and 'eta' in names and 'phi' in names:
            if 'mass' not in names and 'energy' not in names:
                records['mass'] = 0.*dask_akarray['pt']
            return vector.zip(records)

        return ak.zip(records)

        # FIXME: we should probably do an ak.Record using sometjhing along the lines of:
        # ele_rec = ak.zip({'pt': tkele.pt, 'eta': tkele.eta, 'phi': tkele.phi}, with_name="pippo")
        # this would allow to handle the records and assign behaviours....

        # return akarray

# 7. L1Histos

In [16]:
from array import array

import awkward as ak
import hist

# import root_numpy as rnp
import numpy as np
import ROOT

# import pandas as pd
import uproot as up
from scipy.special import expit
import time


stuff = []

class HistoManager:
    class __TheManager:
        def __init__(self):
            self.val = None
            self.histoList = list()
            self.file = None

        def __str__(self):
            return f'self{self.val}'

        def addHistos(self, histo):
            # print 'ADD histo: {}'.format(histo)
            self.histoList.append(histo)

        def writeHistos(self):
            print("WRITING HISTOS [JUPYTER]")
            print("LIST [WRITING HISTOS JUPYTER]:", self.histoList)
            for histo in self.histoList:
                histo.write(self.file)

    instance = None

    def __new__(cls):
        if not HistoManager.instance:
            HistoManager.instance = HistoManager.__TheManager()
        return HistoManager.instance

    def __getattr__(self, name):
        return getattr(self.instance, name)

    def __setattr__(self, name):
        return setattr(self.instance, name)


class BaseHistos:
    def __init__(self, name, root_file=None, debug=False):
        print("INIT BASE HISTOS [JUPYTER]")
        self.name_ = name
        # print name
        # print self.__class__.__name__
        # # print 'BOOK histo: {}'.format(self)
        if root_file is not None:
            root_file.cd()
            # print 'class: {}'.format(self.__class__.__name__)
            # ROOT.gDirectory.pwd()
            file_dir = root_file.GetDirectory(self.__class__.__name__)
            # print '# keys in dir: {}'.format(len(file_dir.GetListOfKeys()))
            # file_dir.cd()
            selhistos = [(histo.ReadObj(), histo.GetName())
                         for histo in file_dir.GetListOfKeys()
                         if histo.GetName().startswith(f'{name}_')]
            if debug:
                print(selhistos)
            for hinst, histo_name in selhistos:
                attr_name = f"h_{histo_name.split(f'{name}_')[1]}"
                setattr(self, attr_name, hinst)
#            self.h_test = root_file.Get('h_EleReso_ptRes')
            # print 'XXXXXX'+str(self.h_test)
        else:
            # for histo in [a for a in dir(self) if a.startswith('h_')]:
                # FIXME
                # getattr(self, histo).Sumw2()
            hm = HistoManager()
            hm.addHistos(self)

    def write(self, upfile):
        dir_name = self.__class__.__name__
        for histo in [a for a in dir(self) if a.startswith('h_')]:
            writeable_hist = getattr(self, histo)
            
            name = writeable_hist.label
            print("COMPUTING JUPYTER: ", writeable_hist.label, " TYPE: ", type(writeable_hist))
            #start_time = time.time()
            writeable_hist = writeable_hist.compute()
            #end_time = time.time()
            #took_time = end_time - start_time
            #print("TOOK: ", took_time, " seconds ")

            # print (f"Writing {histo} class {writeable_hist.__class__.__name__}")
            if 'GraphBuilder' in writeable_hist.__class__.__name__ :
                continue
            elif 'TH1' in writeable_hist.__class__.__name__ or 'TH2' in writeable_hist.__class__.__name__:
                # print('start')
                # FIXME: this somehow fails randomply. ROOT not lining the right python???
                upfile[f'{dir_name}/{writeable_hist.GetName()}'] = writeable_hist
                # print('ok')
            else:
                up_writeable_hist = up.to_writable(writeable_hist)
                upfile[f'{dir_name}/{name}'] = up_writeable_hist

    # def normalize(self, norm):
    #     className = self.__class__.__name__
    #     ret = className()
    #     return ret

    def __repr__(self):
        return f'<{self.__class__.__name__} {self.name_}>'


class GraphBuilder:
    def __init__(self, h_obj, h_name):
        self.h_obj = h_obj
        self.h_name = h_name

    def get_graph(self, name, title, x, y, ex_l, ex_h, ey_l, ey_h):
        global stuff
        ret = ROOT.TGraphAsymmErrors()
        stuff.append(ret)
        ret.SetName(name)
        ret.SetTitle(title)
        ret.Set(len(x))
        for idx in range(len(x)):
            ret.SetPoint(idx, x[idx], y[idx])
            ret.SetPointError(idx, ex_l[idx], ex_h[idx], ey_l[idx], ey_h[idx])
            ret.SetMarkerStyle(2)
        return ret

    def __call__(self, name, title, function):
        h2d = getattr(self.h_obj, self.h_name)
        x, y, ex_l, ex_h, ey_l, ey_h = function(h2d)
        h_title_parts = h2d.GetTitle().split(';')
        g_title = f'{h_title_parts[0]};; {title}'
        if len(h_title_parts) == 2:
            g_title = f'{h_title_parts[0]}; {h_title_parts[1]}; {title}'
        elif len(h_title_parts) == 3:
            g_title += f'{h_title_parts[0]}; {h_title_parts[1]}; {title} ({h_title_parts[2]})'

        graph = self.get_graph(f'{h2d.GetName()}_{name}',
                               g_title, x, y, ex_l, ex_h, ey_l, ey_h)
        g_attr_name = f"g_{self.h_name.split('h_')[1]}_{name}"
        setattr(self.h_obj, g_attr_name, graph)
        return graph

    def Write(self, name='', options=None):
        return


class BaseResoHistos(BaseHistos):
    """

    Base class for resolution histogram classes.

    The class adds a special method to produce a graph out of each
    2D histograms of the class via e special <histoname>_graph method.
    The interface of this method is actually defined by the __call__
    method of the GraphBuilder class.
    If the method is called, the newly created graph is also added permanently
    to the class members and can be reused later.

    Example:
    -------
    def computeEResolution():
        ....
    hreso.h_energyResVenergy_graph('sigmaEOE', '#sigma_{E}/E', computeEResolution)
    will create the graph accessible with:
    hreso.g_energyResVenergy_sigmaEOE
    )
    --------

    """

    def __init__(self, name, root_file=None, debug=False):
        BaseHistos.__init__(self, name, root_file, debug)
        if root_file is not None:
            # print dir(self)
            for attr_2d in [attr for attr in dir(self) if (attr.startswith('h_') and 'TH2' in getattr(self, attr).__class__.__name__)]:
                setattr(self, f'{attr_2d}_graph', GraphBuilder(self, attr_2d))


class BaseUpTuples(BaseHistos):
    def __init__(self, tuple_suffix, name, root_file=None, debug=False):
        self.t_name = f'{name}_{tuple_suffix}'
        self.init_ = False
        # if not root_file:
        #     self.t_name = '{}_{}'.format(name, tuple_suffix)
        if root_file:
            # print(root_file)
            # print(root_file.GetName())
            upfile = up.open(root_file.GetName(), num_workers=1)
            # print(upfile.keys())
            dir_name = self.__class__.__name__
            self.tree = upfile[f'{dir_name}/{self.t_name}']
        BaseHistos.__init__(self, name, root_file, debug)

    def fill(self, data):
        dir_name = self.__class__.__name__
        hm = HistoManager()
        obj_name = f'{dir_name}/{self.t_name}'
        # print( hm.file.keys())
        # print(f'OBJECT: {obj_name}')
        if self.init_:
            # print('extending')
            hm.file[f'{dir_name}/{self.t_name}'].extend(data)
        else:
            # print('creating')
            # hm.file.mktree(f'{dir_name}/{self.t_name}')
            hm.file[f'{dir_name}/{self.t_name}'] = data
            self.init_ = True

    def write(self, upfile):
        return


class BaseTuples(BaseHistos):
    def __init__(self, tuple_suffix, tuple_variables,
                 name, root_file=None, debug=False):
        if not root_file:
            self.t_values = ROOT.TNtuple(
                f'{name}_{tuple_suffix}',
                f'{name}_{tuple_suffix}',
                tuple_variables)
        BaseHistos.__init__(self, name, root_file, debug)

    def write(self):
        if self.__class__.__name__ not in ROOT.gDirectory.GetListOfKeys():
            ROOT.gDirectory.mkdir(self.__class__.__name__)
        newdir = ROOT.gDirectory.GetDirectory(self.__class__.__name__)
        newdir.cd()
        self.t_values.Write()
        ROOT.gDirectory.cd('..')


class CompCatTuples(BaseUpTuples):
    def __init__(self, name, root_file=None, debug=False):
        BaseUpTuples.__init__(
            self, 'CompCatData', name, root_file, debug)

    def fill(self, reference, target):
        # print(self.t_name)
        # print(target.fields)
        target_vars = [
            'pt',
            'eta',
            'phi',
            'compChi2RZ',
            'compChi2RPhi',
            'compChi2Bend',
            'compNstubs',
            'tkPt',
            'compDphi',
            'compDeta',
            'compDpt',
            'compSrrtot',
            'compHoe',
            'compMeanz',
            'compBDTScore']
        rference_vars = [
            'pt',
            'eta',
            'phi',]
        # FIXME: add dz0 gen-track
        tree_data = {}
        for var in target_vars:
            tree_data[var] = ak.flatten(ak.drop_none(target[var]))
        for var in rference_vars:
            tree_data[f'gen_{var}'] = ak.flatten(ak.drop_none(reference[var]))
        # print(reference.fields)
        tree_data['gen_dz'] = ak.flatten(ak.drop_none(np.abs(reference.ovz-target.tkZ0)))

        BaseUpTuples.fill(self, tree_data)


class CompTuples(BaseUpTuples):
    def __init__(self, name, root_file=None, debug=False):
        BaseUpTuples.__init__(
            self, 'CompData', name, root_file, debug)

    def fill(self, data):
# Index(['pt', 'energy', 'eta', 'phi', 'tkIso', 'pfIso', 'puppiIso', 'tkChi2',
#        'tkPt', 'tkZ0', 'compBDTScore', 'compBdt', 'compHoe', 'compSrrtot',
#        'compDeta', 'compDphi', 'compDpt', 'compMeanz', 'compNstubs',
#        'compChi2RPhi', 'compChi2RZ', 'compChi2Bend', 'dpt', 'hwQual',
#        'IDTightSTA', 'IDTightEle', 'IDTightPho', 'IDNoBrem', 'IDBrem'],
#       dtype='object')
        # FIXME: here we do the selection of the tree branches and other manipulations
        vars = [
            'pt',
            'eta',
            'phi',
            'compChi2RZ',
            'compChi2RPhi',
            'compChi2Bend',
            'compNstubs',
            'tkPt',
            'compDphi',
            'compDeta',
            'compDpt',
            'compSrrtot',
            'compHoe',
            'compMeanz',
            'compBDTScore']
        tree_data = {}
        for var in vars:
            if var in data.fields:
                tree_data[var] = data[var]
        BaseUpTuples.fill(self, tree_data)



class HGCIdTuples(BaseUpTuples):
    def __init__(self, name, root_file=None, debug=False):
        BaseUpTuples.__init__(
            self, "CompData", name, root_file, debug)

    def fill(self, data):
# Index(['pt', 'energy', 'eta', 'phi', 'tkIso', 'pfIso', 'puppiIso', 'tkChi2',
#        'tkPt', 'tkZ0', 'compBDTScore', 'compBdt', 'compHoe', 'compSrrtot',
#        'compDeta', 'compDphi', 'compDpt', 'compMeanz', 'compNstubs',
#        'compChi2RPhi', 'compChi2RZ', 'compChi2Bend', 'dpt', 'hwQual',
#        'IDTightSTA', 'IDTightEle', 'IDTightPho', 'IDNoBrem', 'IDBrem'],
#       dtype='object')
        # FIXME: here we do the selection of the tree branches and other manipulations
        vars = ['rho', 'phi', 'eta', 'coreshowerlength', 'ebm0', 'ebm1',
       'firstlayer', 'hbm', 'hwQual', 'maxlayer', 'nTcs', 'showerlength',
       'emax1layers', 'emax3layers', 'emax5layers', 'emaxe', 'eot',
       'first1layers', 'first3layers', 'first5layers', 'firstHcal1layers',
       'firstHcal3layers', 'firstHcal5layers', 'hoe', 'last1layers',
       'last3layers', 'last5layers', 'layer10', 'layer50', 'layer90', 'meanz',
       'ntc67', 'ntc90', 'ptEm', 'seemax', 'seetot', 'sppmax', 'spptot',
       'srrmax', 'srrmean', 'srrtot', 'szz', 'varEtaEta', 'varPhiPhi', 'varRR',
       'varZZ', 'pfPuIdPass', 'pfEmIdPass', 'pfPuIdScore', 'pfEmIdScore',
       'egEmIdScore', 'IDTightEm', 'IDLooseEm', 'eMax']
        tree_data = {}
        for var in vars:
            if var in data.fields:
                tree_data[var] = data[var]
        BaseUpTuples.fill(self, tree_data)

class HGCIdMatchTuples(BaseUpTuples):
    def __init__(self, name, root_file=None, debug=False):
        BaseUpTuples.__init__(
            self, "CompCatData", name, root_file, debug)

    def fill(self, reference, target):
        # print(self.t_name)
        # print(target.fields)
        target_vars = ['rho', 'phi', 'eta', 'coreshowerlength', 'ebm0', 'ebm1',
       'firstlayer', 'hbm', 'hwQual', 'maxlayer', 'nTcs', 'showerlength',
       'emax1layers', 'emax3layers', 'emax5layers', 'emaxe', 'eot',
       'first1layers', 'first3layers', 'first5layers', 'firstHcal1layers',
       'firstHcal3layers', 'firstHcal5layers', 'hoe', 'last1layers',
       'last3layers', 'last5layers', 'layer10', 'layer50', 'layer90', 'meanz',
       'ntc67', 'ntc90', 'ptEm', 'seemax', 'seetot', 'sppmax', 'spptot',
       'srrmax', 'srrmean', 'srrtot', 'szz', 'varEtaEta', 'varPhiPhi', 'varRR',
       'varZZ', 'pfPuIdPass', 'pfEmIdPass', 'pfPuIdScore', 'pfEmIdScore',
       'egEmIdScore', 'IDTightEm', 'IDLooseEm', 'eMax']
        rference_vars = [
            'rho',
            'eta',
            'phi',
            'pdgid',
            'caloeta', 
            'calophi']
        # FIXME: add dz0 gen-track
        tree_data = {}
        for var in target_vars:
            tree_data[var] = ak.flatten(ak.drop_none(target[var]))
        for var in rference_vars:
            tree_data[f'gen_{var}'] = ak.flatten(ak.drop_none(reference[var]))
        # print(reference.fields)
        # tree_data[f'gen_dz'] = ak.flatten(ak.drop_none(np.abs(reference.ovz-target.tkZ0)))
        
        BaseUpTuples.fill(self, tree_data)



class GenPartHistos(BaseHistos):
    def __init__(self, name, root_file=None, debug=False):
        self.h_pt = TH1F(f'{name}_pt', 'Gen Part Pt (GeV)', 100, 0, 100)
        self.h_energy = TH1F(f'{name}_energy', 'Gen Part Energy (GeV)', 100, 0, 1000)
        BaseHistos.__init__(self, name, root_file, debug)

    def fill(self, gps):
        fill_1Dhist(self.h_pt, gps.pt)
        fill_1Dhist(self.h_energy, gps.energy)

    # def write(self):
    #     for histo in [a for a in dir(self) if a.startswith('h_')]:
    #         getattr(self, histo).Write()





class GenParticleHistos(BaseHistos):
    def __init__(self, name, root_file=None, pt_bins=None, debug=False):
        if not root_file:
            self.h_eta = TH1F(f'{name}_eta', 'Gen Part eta; #eta^{GEN};', 50, -3, 3)
            self.h_abseta = TH1F(f'{name}_abseta', 'Gen Part |eta|; |#eta^{GEN}|;', 40, 0, 4)

            if pt_bins is None:
                self.h_pt = TH1F(f'{name}_pt', 'Gen Part P_{T} (GeV); p_{T}^{GEN} [GeV];', 50, 0, 100)
            else:
                self.h_pt = hist.Hist(
                    hist.axis.Variable(pt_bins, name='p_{T}^{GEN} [GeV]'),
                    label=f'{name}_pt',
                    name='Gen Part P_{T} (GeV)',
                    storage=hist.storage.Weight()
                    )
            # print ('bins: {}'.format(pt_bins))
            # print ("# bins: {}".format(n_pt_bins))

        BaseHistos.__init__(self, name, root_file, debug)

    def fill(self, particles):
        weights = None
        if 'weights' in particles.fields:
            weights = particles.weights

        fill_1Dhist(hist=self.h_eta,
                       array=particles.eta)
        fill_1Dhist(hist=self.h_abseta,
                       array=particles.abseta)
        fill_1Dhist(hist=self.h_pt,
                       array=particles.pt)


class GenParticleExtraHistos(GenParticleHistos):
    def __init__(self, name, root_file=None, pt_bins=None, debug=False):
        if not root_file:
            self.h_n = TH1F(f'{name}_#', 'Gen Part #; #', 10, 0, 10)
            self.h_pdgid = TH1F(f'{name}_pdgid', 'Gen Part pdgid; pdgid;', 100, -50, 50)

        GenParticleHistos.__init__(self, name, root_file, pt_bins, debug)

    def fill(self, particles):
        weights = None
        if 'weights' in particles.fields:
            weights = particles.weights

        self.h_n.fill(ak.count(particles.pt, axis=1))
        fill_1Dhist(hist=self.h_pdgid,
                       array=particles.pdgid)


class DigiHistos(BaseHistos):
    def __init__(self, name, root_file=None, debug=False):
        if not root_file:
            self.h_layer = TH1F(f'{name}_layer', 'Digi layer #', 60, 0, 60)
            # self.h_simenergy = TH1F(name+'_energy', 'Digi sim-energy (GeV)', 100, 0, 2)
        BaseHistos.__init__(self, name, root_file, debug)

    def fill(self, digis):
        fill_1Dhist(self.h_layer, digis.layer)
        # rnp.fill_hist(self.h_simenergy, digis.simenergy)


class TCHistos(BaseHistos):
    def __init__(self, name, root_file=None, debug=False):
        if not root_file:
            self.h_energy = TH1F(f'{name}_energy', 'TC energy (GeV)', 100, 0, 2)
            self.h_subdet = TH1F(f'{name}_subdet', 'TC subdet #', 8, 0, 8)
            self.h_mipPt = TH1F(f'{name}_mipPt', 'TC MIP Pt', 50, 0, 10)

            self.h_layer = ROOT.TProfile(f'{name}_layer', 'TC layer #', 60, 0, 60, 's')
            self.h_absz = TH1F(f'{name}_absz', 'TC z(cm)', 100, 300, 500)
            self.h_wafertype = TH1F(f'{name}_wafertype', 'Wafer type', 10, 0, 10)
            self.h_layerVenergy = TH2F(f'{name}_layerVenergy', 'Energy (GeV) vs Layer #', 60, 0, 60, 100, 0, 2)
            self.h_energyVeta = TH2F(f'{name}_energyVeta', 'Energy (GeV) vs Eta', 100, -3.5, 3.5, 100, 0, 2)
            # self.h_energyVetaL1t5 = TH2F(name+'_energyVetaL1t5', "Energy (GeV) vs Eta (layers 1 to 5)", 100, -3.5, 3.5, 100, 0, 2)
            # self.h_energyVetaL6t10 = TH2F(name+'_energyVetaL6t10', "Energy (GeV) vs Eta (layers 6 to 10)", 100, -3.5, 3.5, 100, 0, 2)
            # self.h_energyVetaL11t20 = TH2F(name+'_energyVetaL11t20', "Energy (GeV) vs Eta (layers 11 to 20)", 100, -3.5, 3.5, 100, 0, 2)
            # self.h_energyVetaL21t60 = TH2F(name+'_energyVetaL21t60', "Energy (GeV) vs Eta (layers 21 to 60)", 100, -3.5, 3.5, 100, 0, 2)
            self.h_energyPetaVphi = ROOT.TProfile2D(f'{name}_energyPetaVphi', 'Energy profile (GeV) vs Eta and Phi', 100, -3.5, 3.5, 100, -3.2, 3.2)

        BaseHistos.__init__(self, name, root_file, debug)

    def fill(self, tcs):
        rnp.fill_hist(self.h_energy, tcs.energy)
        rnp.fill_hist(self.h_subdet, tcs.subdet)
        rnp.fill_hist(self.h_mipPt, tcs.mipPt)
        cnt = tcs.layer.value_counts().to_frame(name='counts')
        cnt['layer'] = cnt.index.values
        rnp.fill_profile(self.h_layer, cnt[['layer', 'counts']])
        rnp.fill_hist(self.h_absz, np.fabs(tcs.z))
        rnp.fill_hist(self.h_wafertype, tcs.wafertype)
        rnp.fill_hist(self.h_wafertype, tcs.wafertype)
        # FIXME: should bin this guy in eta bins
        rnp.fill_hist(self.h_layerVenergy, tcs[['layer', 'energy']])
        rnp.fill_hist(self.h_energyVeta, tcs[['eta', 'energy']])
        rnp.fill_hist(self.h_energyVeta, tcs[['eta', 'energy']])
        # rnp.fill_hist(self.h_energyVetaL1t5, tcs[(tcs.layer >= 1) & (tcs.layer <= 5)][['eta', 'energy']])
        # rnp.fill_hist(self.h_energyVetaL6t10, tcs[(tcs.layer >= 6) & (tcs.layer <= 10)][['eta', 'energy']])
        # rnp.fill_hist(self.h_energyVetaL11t20, tcs[(tcs.layer >= 11) & (tcs.layer <= 20)][['eta', 'energy']])
        # rnp.fill_hist(self.h_energyVetaL21t60, tcs[(tcs.layer >= 21) & (tcs.layer <= 60)][['eta', 'energy']])
        rnp.fill_profile(self.h_energyPetaVphi, tcs[['eta', 'phi', 'energy']])


class ClusterHistos(BaseHistos):
    def __init__(self, name, root_file=None, debug=False):
        if not root_file:
            self.h_energy = TH1F(f'{name}_energy', 'Cluster energy (GeV); E [GeV];', 100, 0, 30)
            self.h_layer = TH1F(f'{name}_layer', 'Cluster layer #; layer #;', 60, 0, 60)
            # self.h_nCoreCells = TH1F(name+'_nCoreCells', 'Cluster # cells (core)', 30, 0, 30)

            self.h_layerVenergy = TH2F(f'{name}_layerVenergy', 'Cluster Energy (GeV) vs Layer #; layer; E [GeV];', 50, 0, 50, 100, 0, 20)
            self.h_ncells = TH1F(f'{name}_ncells', 'Cluster # cells; # TC components;', 30, 0, 30)
            self.h_layerVncells = TH2F(f'{name}_layerVncells', 'Cluster #cells vs Layer #; layer; # TC components;',  50, 0, 50, 30, 0, 30)
            # self.h_layerVnCoreCells = TH2F(name+'_layerVnCoreCells', "Cluster #cells vs Layer #",  50, 0, 50, 30, 0, 30)

        BaseHistos.__init__(self, name, root_file, debug)

    def fill(self, clsts):
        rnp.fill_hist(self.h_energy, clsts.energy)
        rnp.fill_hist(self.h_layer, clsts.layer)
        rnp.fill_hist(self.h_layerVenergy, clsts[['layer', 'energy']])
        # if 'ncells' in clsts.columns:
        rnp.fill_hist(self.h_ncells, clsts.ncells)
        rnp.fill_hist(self.h_layerVncells, clsts[['layer', 'ncells']])
        # if 'nCoreCells' in clsts.columns:
        #     rnp.fill_hist(self.h_nCoreCells, clsts.nCoreCells)
        #     rnp.fill_hist(self.h_layerVnCoreCells, clsts[['layer', 'nCoreCells']])


class Cluster3DHistos(BaseHistos):
    def __init__(self, name, root_file=None, debug=False):
        if not root_file:
            # self.h_npt05 = TH1F(
            #     name+'_npt05', '# 3D Cluster Pt > 0.5 GeV; # 3D clusters in cone;', 1000, 0, 1000)
            # self.h_npt20 = TH1F(
            #     name+'_npt20', '# 3D Cluster Pt > 2.0 GeV; # 3D clusters in cone;', 1000, 0, 1000)
            self.h_pt = TH1F(
                f'{name}_pt', '3D Cluster Pt (GeV); p_{T} [GeV]', 100, 0, 100)
            self.h_eta = TH1F(f'{name}_eta', '3D Cluster eta; #eta;', 100, -4, 4)
            # self.h_energy = TH1F(name+'_energy', '3D Cluster energy (GeV); E [GeV]', 1000, 0, 1000)
            # self.h_nclu = TH1F(name+'_nclu', '3D Cluster # clusters; # 2D components;', 60, 0, 60)
            # self.h_ncluVpt = TH2F(name+'_ncluVpt', '3D Cluster # clusters vs pt; # 2D components; p_{T} [GeV]', 60, 0, 60, 100, 0, 100)
            # self.h_showlenght = TH1F(name+'_showlenght', '3D Cluster showerlenght', 60, 0, 60)
            # self.h_firstlayer = TH1F(name+'_firstlayer', '3D Cluster first layer', 30, 0, 30)
            # self.h_sEtaEtaTot = TH1F(name+'_sEtaEtaTot', '3D Cluster RMS Eta', 100, 0, 0.1)
            # self.h_sEtaEtaMax = TH1F(name+'_sEtaEtaMax', '3D Cluster RMS Eta (max)', 100, 0, 0.1)
            # self.h_sPhiPhiTot = TH1F(name+'_sPhiPhiTot', '3D Cluster RMS Phi', 100, 0, 2)
            # self.h_sPhiPhiMax = TH1F(name+'_sPhiPhiMax', '3D Cluster RMS Phi (max)', 100, 0, 2)
            # self.h_sZZ = TH1F(name+'_sZZ', '3D Cluster RMS Z ???', 100, 0, 10)
            # self.h_eMaxOverE = TH1F(name+'_eMaxOverE', '3D Cluster Emax/E', 100, 0, 1)
            # self.h_HoE = TH1F(name+'_HoE', '3D Cluster H/E', 20, 0, 2)
            # self.h_iso0p2 = TH1F(name+'_iso0p2', '3D Cluster iso DR 0.2(GeV); Iso p_{T} [GeV];', 100, 0, 100)
            # self.h_isoRel0p2 = TH1F(name+'_isoRel0p2', '3D Cluster relative iso DR 0.2; Rel. Iso;', 100, 0, 1)
            # self.h_bdtPU = TH1F(name+'_bdtPU', '3D Cluster bdt PU out; BDT-PU out;', 100, -1, 1)
            # self.h_bdtPi = TH1F(name+'_bdtPi', '3D Cluster bdt Pi out; BDT-Pi out;', 100, -1, 1)
            # self.h_bdtEg = TH1F(name+'_bdtEg', '3D Cluster bdt Pi out; BDT-EG out;', 100, -1, 1)

        BaseHistos.__init__(self, name, root_file, debug)

    def fill(self, cl3ds):
        # self.h_npt05.Fill(len(cl3ds[cl3ds.pt > 0.5].index))
        # self.h_npt20.Fill(len(cl3ds[cl3ds.pt > 2.0].index))

        fill_1Dhist(self.h_pt, cl3ds.pt)
        fill_1Dhist(self.h_eta, cl3ds.eta)
        # fill_1Dhist(self.h_energy, cl3ds.energy)
        # fill_1Dhist(self.h_nclu, cl3ds.nclu)
        # fill_2Dhist(self.h_ncluVpt, cl3ds[['nclu', 'pt']])
        # fill_1Dhist(self.h_showlenght, cl3ds.showerlength)
        # fill_1Dhist(self.h_firstlayer, cl3ds.firstlayer)
        # fill_1Dhist(self.h_sEtaEtaTot, cl3ds.seetot)
        # fill_1Dhist(self.h_sEtaEtaMax, cl3ds.seemax)
        # fill_1Dhist(self.h_sPhiPhiTot, cl3ds.spptot)
        # fill_1Dhist(self.h_sPhiPhiMax, cl3ds.sppmax)
        # fill_1Dhist(self.h_sZZ, cl3ds.szz)
        # fill_1Dhist(self.h_eMaxOverE, cl3ds.emaxe)
        # fill_1Dhist(self.h_HoE, cl3ds.hoe)
        # if 'iso0p2' in cl3ds.fields:
        #     fill_1Dhist(self.h_iso0p2, cl3ds.iso0p2)
        #     fill_1Dhist(self.h_isoRel0p2, cl3ds.isoRel0p2)
        # if 'bdt_pu' in cl3ds.fields:
        #     fill_1Dhist(self.h_bdtPU, cl3ds.bdt_pu)
        # if 'bdt_pi' in cl3ds.fields:
        #     fill_1Dhist(self.h_bdtPi, cl3ds.bdt_pi)
        # fill_1Dhist(self.h_bdtEg, cl3ds.bdteg)


class EGHistos(BaseHistos):
    def __init__(self, name, root_file=None, debug=False):
        if not root_file:
            self.h_pt = TH1F(f'{name}_pt', 'EG Pt (GeV); p_{T} [GeV]', 100, 0, 100)
            self.h_eta = TH1F(f'{name}_eta', 'EG eta; #eta;', 100, -4, 4)
            self.h_energy = TH1F(f'{name}_energy', 'EG energy (GeV); E [GeV]', 1000, 0, 1000)
            self.h_hwQual = TH1F(f'{name}_hwQual', 'EG energy (GeV); hwQual', 5, 0, 5)
            self.h_tkIso = TH1F(f'{name}_tkIso', 'Iso; rel-iso_{tk}', 100, 0, 2)
            self.h_pfIso = TH1F(f'{name}_pfIso', 'Iso; rel-iso_{pf}', 100, 0, 2)
            self.h_tkIsoPV = TH1F(f'{name}_tkIsoPV', 'Iso; rel-iso^{PV}_{tk}', 100, 0, 2)
            self.h_pfIsoPV = TH1F(f'{name}_pfIsoPV', 'Iso; rel-iso^{PV}_{pf}', 100, 0, 2)
            self.h_n = TH1F(f'{name}_n', '# objects per event', 100, 0, 100)
            self.h_compBdt = TH1F(f'{name}_compBdt', 'BDT Score Comp ID', 50, 0, 1)

        BaseHistos.__init__(self, name, root_file, debug)

    def fill(self, egs):
        print("FILLING JUPYTER")
        weight = None
        if 'weight' in egs.fields:
            weight = egs.weight

        fill_1Dhist(hist=self.h_pt,     array=egs.pt,     weights=weight)
        fill_1Dhist(hist=self.h_eta,    array=egs.eta,    weights=weight)
        # fill_1Dhist(hist=self.h_energy, array=egs.energy, weights=weight)
        fill_1Dhist(hist=self.h_hwQual, array=egs.hwQual, weights=weight)
        if 'tkIso' in egs.fields:
            fill_1Dhist(hist=self.h_tkIso, array=egs.tkIso, weights=weight)
        if 'pfIso' in egs.fields:
            fill_1Dhist(hist=self.h_pfIso, array=egs.pfIso, weights=weight)
        if 'tkIsoPV' in egs.fields:
            fill_1Dhist(hist=self.h_tkIsoPV, array=egs.tkIsoPV, weights=weight)
            fill_1Dhist(hist=self.h_pfIsoPV, array=egs.pfIsoPV, weights=weight)
        if 'compBDTScore' in egs.fields:
            fill_1Dhist(hist=self.h_compBdt, array=egs.compBDTScore, weights=weight)
        if 'idScore' in egs.fields:
            fill_1Dhist(hist=self.h_compBdt, array=expit(egs.idScore), weights=weight)
        # print(ak.count(egs.pt, axis=1))
        # print(egs.pt.type.show())
        # print(ak.count(egs.pt, axis=1).type.show())
        self.h_n.fill(ak.count(egs.pt, axis=1))
        # fill_1Dhist(hist=self.h_n, array=ak.count(egs.pt, axis=1), weights=weight)
        # self.h_n.Fill()


    def add_histos(self):
        self.h_pt.Add(self.h_pt_temp.GetValue())
        self.h_eta.Add(self.h_eta_temp.GetValue())
        self.h_energy.Add(self.h_energy_temp.GetValue())
        self.h_hwQual.Add(self.h_hwQual_temp.GetValue())
        # self.h_tkIso = TH1F(name+'_tkIso', 'Iso; rel-iso_{tk}', 100, 0, 2)
        # self.h_pfIso = TH1F(name+'_pfIso', 'Iso; rel-iso_{pf}', 100, 0, 2)
        # self.h_tkIsoPV = TH1F(name+'_tkIsoPV', 'Iso; rel-iso^{PV}_{tk}', 100, 0, 2)
        # self.h_pfIsoPV = TH1F(name+'_pfIsoPV', 'Iso; rel-iso^{PV}_{pf}', 100, 0, 2)


class DecTkHistos(BaseHistos):
    def __init__(self, name, root_file=None, debug=False):
        if not root_file:
            self.h_pt = TH1F(
                f'{name}_pt',
                'Pt (GeV); p_{T} [GeV]',
                100, 0, 100)
            self.h_deltaPt = TH1F(
                f'{name}_deltaPt',
                'Pt (GeV); p_{T}^{decoded}-p_{T}^{float}  [GeV]',
                100, -10, 10)
            self.h_deltaPtVeta = TH2F(
                f'{name}_deltaPtVeta',
                'Pt (GeV); #eta^{float}; p_{T}^{decoded}-p_{T}^{float}  [GeV]',
                50, -2.5, 2.5,
                50, -0.25, 0.25)
            self.h_deltaPtVabseta = TH2F(
                f'{name}_deltaPtVabseta',
                'Pt (GeV); |#eta^{float}|; p_{T}^{decoded}-p_{T}^{float}  [GeV]',
                50, 0, 2.5,
                50, -0.25, 0.25)
            self.h_eta = TH1F(
                f'{name}_eta',
                '#eta; #eta;',
                100, -4, 4)
            self.h_z0 = TH1F(
                f'{name}_z0',
                'z0; z_{0} [cm];',
                100, -10, 10)
            self.h_deltaZ0 = TH1F(
                f'{name}_deltaZ0',
                '#Delta z0; z0^{decoded}-z0^{float};',
                50, -0.2, 0.2)
            self.h_deltaZ0Veta = TH2F(
                f'{name}_deltaZ0Veta',
                '#Delta z0; #eta^{float}; z0^{decoded}-z0^{float};',
                100, -2.5, 2.5,
                50, -0.05, 0.05)
            self.h_deltaEta = TH1F(
                f'{name}_deltaEta',
                '#Delta #eta_{@vtx}; #eta_{@vtx}^{decoded}-#eta_{@vtx}^{float};',
                100, -1, 1)
            self.h_deltaEtaVabseta = TH2F(
                f'{name}_deltaEtaVabseta',
                '#Delta #eta_{@vtx} vs |#eta^{float}|; |#eta^{float}|; #eta_{@vtx}^{decoded}-#eta_{@vtx}^{float};',
                25, 0, 2.5,
                100, -0.004, 0.004)
            self.h_deltaEtaVeta = TH2F(
                f'{name}_deltaEtaVeta',
                '#Delta #eta_{@vtx} vs #eta^{float}; #eta^{float}; #eta_{@vtx}^{decoded}-#eta_{@vtx}^{float};',
                50, -2.5, 2.5,
                50, -0.004, 0.004)
            self.h_deltaCaloEta = TH1F(
                f'{name}_deltaCaloEta',
                '#Delta #eta_{@calo}; #eta_{@calo}^{decoded}-#eta_{@calo}^{float};',
                100, -1, 1)
            self.h_deltaCaloEtaVabseta = TH2F(
                f'{name}_deltaCaloEtaVabseta',
                '#Delta #eta_{@calo} vs |#eta^{float}|; |#eta^{float}|; #eta_{@calo}^{decoded}-#eta_{@calo}^{float};',
                50, 0, 2.5,
                100, -0.04, 0.04)
            self.h_deltaCaloEtaVeta = TH2F(
                f'{name}_deltaCaloEtaVeta',
                '#Delta #eta_{@calo} vs #eta^{float}; #eta^{float}; #eta_{@calo}^{decoded}-#eta_{@calo}^{float};',
                100, -2.5, 2.5,
                100, -0.04, 0.04)
            self.h_deltaCaloPhi = TH1F(
                f'{name}_deltaCaloPhi',
                '#Delta #phi_{@calo}; #phi_{@calo}^{decoded}-#phi_{@calo}^{float};',
                100, -1, 1)
            self.h_deltaCaloPhiVabseta = TH2F(
                f'{name}_deltaCaloPhiVabseta',
                '#Delta #phi_{@calo} vs |#eta^{float}|; |#phi^{float}|; #phi_{@calo}^{decoded}-#phi_{@calo}^{float};',
                100, 0, 2.5,
                100, -0.1, 0.1)

        BaseHistos.__init__(self, name, root_file, debug)

    def fill(self, egs):
        fill_1Dhist(self.h_pt, egs.pt)
        fill_1Dhist(self.h_deltaPt, egs.deltaPt)
        fill_2Dhist(self.h_deltaPtVeta, egs.simeta, egs.deltaPt)
        fill_2Dhist(self.h_deltaPtVabseta, egs.simabseta, egs.deltaPt)
        fill_1Dhist(self.h_eta, egs.eta)
        fill_1Dhist(self.h_z0, egs.z0)
        fill_1Dhist(self.h_deltaZ0, egs.deltaZ0)
        fill_2Dhist(self.h_deltaZ0Veta, egs.simeta, egs.deltaZ0)
        fill_1Dhist(self.h_deltaEta, egs.deltaEta)
        fill_2Dhist(self.h_deltaEtaVabseta, egs.simabseta, egs.deltaEta)
        fill_2Dhist(self.h_deltaEtaVeta, egs.simeta, egs.deltaEta)
        fill_1Dhist(self.h_deltaCaloEta, egs.deltaCaloEta)
        fill_2Dhist(self.h_deltaCaloEtaVabseta, egs.simabseta, egs.deltaCaloEta)
        fill_2Dhist(self.h_deltaCaloEtaVeta, egs.simeta, egs.deltaCaloEta)
        fill_1Dhist(self.h_deltaCaloPhi, egs.deltaCaloPhi)
        fill_2Dhist(self.h_deltaCaloPhiVabseta, egs.simabseta, egs.deltaCaloPhi)


class TkEleHistos(BaseHistos):
    def __init__(self, name, root_file=None, debug=False):
        if not root_file:
            self.h_pt = TH1F(f'{name}_pt', 'Pt (GeV); p_{T} [GeV]', 100, 0, 100)
            self.h_eta = TH1F(f'{name}_eta', 'eta; #eta;', 100, -2.5, 2.5)
            self.h_energy = TH1F(f'{name}_energy', 'energy (GeV); E [GeV]', 1000, 0, 1000)
            self.h_hwQual = TH1F(f'{name}_hwQual', 'quality; hwQual', 10, 0, 10)
            self.h_tkpt = TH1F(f'{name}_tkpt', 'Tk Pt (GeV); p_{T}^{L1Tk} [GeV]', 100, 0, 100)
            self.h_dpt = TH1F(f'{name}_dpt', 'Delta Tk Pt (GeV); #Delta p_{T}^{L1Tk-Calo} [GeV]', 100, -50, 50)
            self.h_tkchi2 = TH1F(f'{name}_tkchi2', 'Tk chi2; #Chi^{2}', 1000, 0, 1000)
            self.h_ptVtkpt = TH2F(f'{name}_ptVtkpt', 'TkEG Pt (GeV) vs TkPt; p_{T}^{Tk} [GeV]; p_{T}^{EG} [GeV]', 100, 0, 100, 100, 0, 100)
            self.h_tkIso = TH1F(f'{name}_tkIso', 'Iso; rel-iso_{tk}', 100, 0, 2)
            self.h_pfIso = TH1F(f'{name}_pfIso', 'Iso; rel-iso_{pf}', 100, 0, 2)

        BaseHistos.__init__(self, name, root_file, debug)

    def fill(self, tkegs):
        fill_1Dhist(self.h_pt, tkegs.pt)
        fill_1Dhist(self.h_eta, tkegs.eta)
        fill_1Dhist(self.h_energy, tkegs.energy)
        fill_1Dhist(self.h_hwQual, tkegs.hwQual)
        fill_1Dhist(self.h_tkpt, tkegs.tkPt)
        fill_1Dhist(self.h_dpt, tkegs.dpt)
        fill_1Dhist(self.h_tkchi2, tkegs.tkChi2)
        fill_1Dhist(self.h_tkIso, tkegs.tkIso)
        fill_1Dhist(self.h_pfIso, tkegs.pfIso)



class TkEmHistos(BaseHistos):
    def __init__(self, name, root_file=None, debug=False):
        if not root_file:
            self.h_pt = TH1F(f'{name}_pt', 'Pt (GeV); p_{T} [GeV]', 100, 0, 100)
            self.h_eta = TH1F(f'{name}_eta', 'eta; #eta;', 100, -2.5, 2.5)
            self.h_energy = TH1F(f'{name}_energy', 'energy (GeV); E [GeV]', 1000, 0, 1000)
            self.h_hwQual = TH1F(f'{name}_hwQual', 'quality; hwQual', 10, 0, 10)
            self.h_tkIso = TH1F(f'{name}_tkIso', 'Iso; rel-iso_{tk}', 100, 0, 2)
            self.h_pfIso = TH1F(f'{name}_pfIso', 'Iso; rel-iso_{pf}', 100, 0, 2)
            self.h_tkIsoPV = TH1F(f'{name}_tkIsoPV', 'Iso; rel-iso^{PV}_{tk}', 100, 0, 2)
            self.h_pfIsoPV = TH1F(f'{name}_pfIsoPV', 'Iso; rel-iso^{PV}_{pf}', 100, 0, 2)

        BaseHistos.__init__(self, name, root_file, debug)

    def fill(self, tkegs):
        fill_1Dhist(self.h_pt, tkegs.pt)
        fill_1Dhist(self.h_eta, tkegs.eta)
        fill_1Dhist(self.h_energy, tkegs.energy)
        fill_1Dhist(self.h_hwQual, tkegs.hwQual)
        fill_1Dhist(self.h_tkIso, tkegs.tkIso)
        fill_1Dhist(self.h_pfIso, tkegs.pfIso)
        fill_1Dhist(self.h_tkIsoPV, tkegs.tkIsoPV)
        fill_1Dhist(self.h_pfIsoPV, tkegs.pfIsoPV)


class TkEGHistos(BaseHistos):
    def __init__(self, name, root_file=None, debug=False):
        if not root_file:
            self.h_pt = TH1F(f'{name}_pt', 'TkEG Pt (GeV); p_{T} [GeV]', 100, 0, 100)
            self.h_eta = TH1F(f'{name}_eta', 'TkEG eta; #eta;', 100, -4, 4)
            self.h_energy = TH1F(f'{name}_energy', 'TkEG energy (GeV); E [GeV]', 1000, 0, 1000)
            self.h_hwQual = TH1F(f'{name}_hwQual', 'TkEG energy (GeV); hwQual', 5, 0, 5)

            self.h_tkpt = TH1F(f'{name}_tkpt', 'TkEG Pt (GeV); p_{T} [GeV]', 100, 0, 100)
            self.h_tketa = TH1F(f'{name}_tketa', 'TkEG eta; #eta;', 100, -4, 4)
            self.h_tkchi2 = TH1F(f'{name}_tkchi2', 'TkEG chi2; #Chi^{2}', 1000, 0, 1000)
            self.h_tkchi2Red = TH1F(f'{name}_tkchi2Red', 'TkEG chi2 red; reduced #Chi^{2}', 100, 0, 100)
            self.h_tknstubs = TH1F(f'{name}_tknstubs', 'TkEG # stubs; # stubs', 10, 0, 10)
            self.h_tkz0 = TH1F(f'{name}_tkz0', 'TkEG z0; z_{0} [cm]', 100, -10, 10)
            self.h_tkchi2RedVeta = TH2F(f'{name}_tkchi2RedVeta', 'TkEG chi2 red. v eta; #eta; red. #Chi^{2}', 100, -4, 4, 100, 0, 100)
            self.h_tknstubsVeta = TH2F(f'{name}_tknstubsVeta', 'TkEG # stubs vs eta; #eta; # stubs', 100, -4, 4, 10, 0, 10)
            self.h_tkz0Veta = TH2F(f'{name}_tkz0Veta', 'TkEG z0 vs eta; #eta; z_{0} [cm]', 100, -4, 4, 100, -10, 10)
            self.h_dphi = TH1F(f'{name}_dphi', 'TkEG #Delta #phi; #Delta #phi [rad]', 100, -0.2, 0.2)
            self.h_dphiVpt = TH2F(f'{name}_dphiVpt', 'TkEG #Delta #phi vs p_{T}^{EG}; p_{T}^{EG} [GeV]; #Delta #phi [rad]', 100, 0, 100, 100, -0.2, 0.2)
            self.h_deta = TH1F(f'{name}_deta', 'TkEG #Delta #eta; #Delta #eta', 100, -0.2, 0.2)
            self.h_detaVpt = TH2F(f'{name}_detaVpt', 'TkEG #Delta #eta vs p_{T}^{EG}; p_{T}^{EG} [GeV]; #Delta #eta', 100, 0, 100, 100, -0.2, 0.2)

            self.h_dr = TH1F(f'{name}_dr', 'TkEG #Delta R; #Delta R', 100, 0, 0.2)
            self.h_ptVtkpt = TH2F(f'{name}_ptVtkpt', 'TkEG Pt (GeV) vs TkPt; p_{T}^{Tk} [GeV]; p_{T}^{EG} [GeV]', 100, 0, 100, 100, 0, 100)

        BaseHistos.__init__(self, name, root_file, debug)

    def fill(self, tkegs):
        rnp.fill_hist(self.h_pt, tkegs.pt)
        rnp.fill_hist(self.h_eta, tkegs.eta)
        rnp.fill_hist(self.h_energy, tkegs.energy)
        rnp.fill_hist(self.h_hwQual, tkegs.hwQual)
        rnp.fill_hist(self.h_tkpt, tkegs.tkpt)
        rnp.fill_hist(self.h_tketa, tkegs.tketa)
        rnp.fill_hist(self.h_tkchi2, tkegs.tkchi2)
        rnp.fill_hist(self.h_tkchi2Red, tkegs.tkchi2Red)
        rnp.fill_hist(self.h_tknstubs, tkegs.tknstubs)
        rnp.fill_hist(self.h_tkz0, tkegs.tkz0)
        rnp.fill_hist(self.h_tkchi2RedVeta, tkegs[['eta', 'tkchi2Red']])
        rnp.fill_hist(self.h_tknstubsVeta, tkegs[['eta', 'tknstubs']])
        rnp.fill_hist(self.h_tkz0Veta, tkegs[['eta', 'tkz0']])
        rnp.fill_hist(self.h_dphi, tkegs.dphi)
        rnp.fill_hist(self.h_deta, tkegs.deta)
        rnp.fill_hist(self.h_dphiVpt, tkegs[['pt', 'dphi']])
        rnp.fill_hist(self.h_detaVpt, tkegs[['pt', 'deta']])
        rnp.fill_hist(self.h_dr, tkegs.dr)
        rnp.fill_hist(self.h_ptVtkpt, tkegs[['tkpt', 'pt']])


class TrackHistos(BaseHistos):
    def __init__(self, name, root_file=None, debug=False):
        if not root_file:
            self.h_pt = TH1F(f'{name}_pt',
                                'Track Pt (GeV); p_{T} [GeV]', 100, 0, 100)
            self.h_eta = TH1F(f'{name}_eta',
                                 'Track eta; #eta;', 100, -4, 4)
            self.h_chi2 = TH1F(f'{name}_chi2',
                                  'Track chi2; #Chi^{2}', 1000, 0, 1000)
            self.h_chi2Red = TH1F(f'{name}_chi2Red',
                                     'Track chi2 red; red. #Chi^{2}', 100, 0, 100)
            self.h_nstubs = TH1F(f'{name}_nstubs',
                                    'Track # stubs; # stubs', 10, 0, 10)
            self.h_z0 = TH1F(f'{name}_z0',
                                'Track z0; z_{0} [cm]', 100, -10, 10)
            self.h_chi2RedVeta = TH2F(f'{name}_chi2RedVeta',
                                         'Track chi2 red. v eta; #eta; red. #Chi^{2}', 100, -4, 4, 100, 0, 100)
            self.h_nstubsVeta = TH2F(f'{name}_nstubsVeta',
                                        'Track # stubs vs eta; #eta; # stubs', 100, -4, 4, 10, 0, 10)
            self.h_z0Veta = TH2F(f'{name}_z0Veta',
                                    'Track z0 vs eta; #eta; z_{0} [cm]', 100, -4, 4, 100, -10, 10)
            self.h_chi2RedVpt = TH2F(f'{name}_chi2RedVpt',
                                        'Track chi2 red. v pT; p_{T} [GeV]; red. #Chi^{2}', 100, 0, 100, 100, 0, 100)
            self.h_nstubsVpt = TH2F(f'{name}_nstubsVpt',
                                       'Track # stubs vs pT; p_{T} [GeV]; # stubs', 100, 0, 100, 10, 0, 10)
            self.h_z0Vpt = TH2F(f'{name}_z0Vpt',
                                   'Track z0 vs pT; p_{T} [GeV]; z_{0} [cm]', 100, 0, 100, 100, -10, 10)

        BaseHistos.__init__(self, name, root_file, debug)

    def fill(self, tracks):
        fill_1Dhist(self.h_pt, tracks.pt)
        fill_1Dhist(self.h_eta, tracks.eta)
        fill_1Dhist(self.h_chi2, tracks.chi2)
        fill_1Dhist(self.h_chi2Red, tracks.chi2Red)
        fill_1Dhist(self.h_nstubs, tracks.nStubs)
        fill_1Dhist(self.h_z0, tracks.z0)
        fill_2Dhist(self.h_chi2RedVeta, tracks.eta, tracks.chi2Red)
        fill_2Dhist(self.h_nstubsVeta, tracks.eta, tracks.nStubs)
        fill_2Dhist(self.h_z0Veta, tracks.eta, tracks.z0)
        fill_2Dhist(self.h_chi2RedVpt, tracks.pt, tracks.chi2Red)
        fill_2Dhist(self.h_nstubsVpt, tracks.pt, tracks.nStubs)
        fill_2Dhist(self.h_z0Vpt, tracks.pt, tracks.z0)



class JetHistos(BaseHistos):
    def __init__(self, name, root_file=None, debug=False):
        if not root_file:
            self.h_pt = TH1F(f'{name}_pt',
                                'Track Pt (GeV); p_{T} [GeV]', 100, 0, 100)
            self.h_eta = TH1F(f'{name}_eta',
                                 'Track eta; #eta;', 100, -4, 4)

        BaseHistos.__init__(self, name, root_file, debug)

    def fill(self, tracks):
        fill_1Dhist(self.h_pt, tracks.pt)
        fill_1Dhist(self.h_eta, tracks.eta)




class TriggerTowerHistos(BaseHistos):
    def __init__(self, name, root_file=None, debug=False):
        if not root_file:
            self.h_pt = TH1F(f'{name}_pt', 'Tower Pt (GeV); p_{T} [GeV];', 100, 0, 100)
            self.h_etEm = TH1F(f'{name}_etEm', 'Tower Et EM (GeV)', 100, 0, 100)
            self.h_etHad = TH1F(f'{name}_etHad', 'Tower Et Had (GeV)', 100, 0, 100)
            self.h_HoE = TH1F(f'{name}_HoE', 'Tower H/E', 20, 0, 2)
            self.h_HoEVpt = TH2F(f'{name}_HoEVpt', 'Tower H/E vs Pt (GeV); H/E;', 50, 0, 100, 20, 0, 2)
            self.h_energy = TH1F(f'{name}_energy', 'Tower energy (GeV)', 1000, 0, 1000)
            self.h_eta = TH1F(f'{name}_eta', 'Tower eta; #eta;', 75, -3.169, 3.169)
            self.h_ieta = TH1F(f'{name}_ieta', 'Tower eta; i#eta;', 18, 0, 18)

            self.h_ptVeta = TH2F(f'{name}_ptVeta', 'Tower P_P{T} (GeV) vs #eta; #eta; p_{T} [GeV];',  75, -3.169, 3.169, 100, 0, 10)
            self.h_etVieta = TH2F(f'{name}_etVieta', 'Tower E_{T} (GeV) vs ieta; i#eta; E_{T} [GeV];',  18, 0, 18, 100, 0, 10)
            self.h_etEmVieta = TH2F(f'{name}_etEmVieta', 'Tower E_{T} EM (GeV) vs ieta; i#eta; E_{T}^{EM} [GeV];',  18, 0, 18, 100, 0, 10)
            self.h_etHadVieta = TH2F(f'{name}_etHadVieta', 'Tower E_{T} Had (GeV) vs ieta; i#eta; E_{T}^{HAD} [GeV];',  18, 0, 18, 100, 0, 10)
            self.h_sumEt = TH1F(f'{name}_sumEt', 'Tower SumEt (GeV); E_{T}^{TOT} [GeV];', 200, 0, 400)
            self.h_sumEtCentral = TH1F(f'{name}_sumEtCentral', 'Tower SumEt (GeV) (central); E_{T}^{TOT} [GeV];', 200, 0, 400)

        BaseHistos.__init__(self, name, root_file, debug)

    def fill(self, towers):
        rnp.fill_hist(self.h_pt, towers.pt)
        rnp.fill_hist(self.h_etEm, towers.etEm)
        rnp.fill_hist(self.h_etHad, towers.etHad)
        rnp.fill_hist(self.h_HoE, towers.HoE)
        rnp.fill_hist(self.h_HoEVpt, towers[['pt', 'HoE']])
        rnp.fill_hist(self.h_energy, towers.energy)
        rnp.fill_hist(self.h_eta, towers.eta)
        rnp.fill_hist(self.h_ieta, towers.iEta)
        rnp.fill_hist(self.h_ptVeta, towers[['eta', 'pt']])
        rnp.fill_hist(self.h_etVieta, towers[['iEta', 'pt']])
        rnp.fill_hist(self.h_etEmVieta, towers[['iEta', 'etEm']])
        rnp.fill_hist(self.h_etHadVieta, towers[['iEta', 'etHad']])
        if not towers.empty:
            self.h_sumEt.Fill(towers.momentum.sum().Pt())
            central_towers = towers[(towers.iEta != 0) & (towers.iEta != 17)]
            if not central_towers.empty:
                self.h_sumEtCentral.Fill(central_towers.momentum.sum().Pt())


class TriggerTowerResoHistos(BaseResoHistos):
    def __init__(self, name, root_file=None):
        if not root_file:
            self.h_ptRes = TH1F(f'{name}_ptRes', 'TT Pt reso (GeV); (p_{T}^{L1}-p_{T}^{GEN})/p_{T}^{GEN};', 100, -1, 2)

            self.h_ptResVpt = TH2F(f'{name}_ptResVpt', 'TT Pt reso (GeV) vs pt (GeV); p_{T}^{GEN} [GeV]; (p_{T}^{L1}-p_{T}^{GEN})/p_{T}^{GEN};', 50, 0, 100, 100, -1, 2)
            self.h_ptResVeta = TH2F(f'{name}_ptResVeta', 'TT Pt reso (GeV) vs eta; #eta^{GEN}; (p_{T}^{L1}-p_{T}^{GEN})/p_{T}^{GEN};', 100, -3.5, 3.5, 100, -1, 2)

            self.h_ptResp = TH1F(f'{name}_ptResp', 'TT Pt resp.; p_{T}^{L1}/p_{T}^{GEN};', 100, 0, 2)
            self.h_ptRespVpt = TH2F(f'{name}_ptRespVpt', 'TT Pt resp. vs pt (GeV); p_{T}^{GEN} [GeV]; p_{T}^{L1}/p_{T}^{GEN};', 50, 0, 100, 100, 0, 2)
            self.h_ptRespVeta = TH2F(f'{name}_ptRespVeta', 'TT Pt resp. vs |#eta|; |#eta^{GEN}|; p_{T}^{L1}/p_{T}^{GEN};', 34, 1.4, 3.1, 100, 0, 2)

            self.h_energyRes = TH1F(f'{name}_energyRes', 'TT Energy reso (GeV)', 200, -100, 100)
            self.h_energyResVeta = TH2F(f'{name}_energyResVeta', 'TT E reso (GeV) vs eta', 100, -3.5, 3.5, 200, -100, 100)
            # FIXME: add corresponding Pt plots
            self.h_etaRes = TH1F(f'{name}_etaRes', 'TT eta reso; #eta^{L1}-#eta^{GEN}', 100, -0.4, 0.4)
            self.h_phiRes = TH1F(f'{name}_phiRes', 'TT phi reso; #phi^{L1}-#phi^{GEN}', 100, -0.4, 0.4)
            self.h_etalwRes = TH1F(f'{name}_etalwRes', 'TT eta reso (lw)', 100, -0.4, 0.4)
            self.h_philwRes = TH1F(f'{name}_philwRes', 'TT phi reso (lw)', 100, -0.4, 0.4)

            self.h_drRes = TH1F(f'{name}_drRes', 'TT DR reso', 100, 0, 0.1)
        BaseResoHistos.__init__(self, name, root_file)

    def fill(self, reference, target):
        self.h_ptRes.Fill((target.pt - reference.pt)/reference.pt)
        self.h_ptResVpt.Fill(reference.pt, (target.pt - reference.pt)/reference.pt)
        self.h_ptResVeta.Fill(reference.eta, (target.pt - reference.pt)/reference.pt)

        self.h_ptResp.Fill(target.pt/reference.pt)
        self.h_ptRespVpt.Fill(reference.pt, target.pt/reference.pt)
        self.h_ptRespVeta.Fill(abs(reference.eta), target.pt/reference.pt)

        self.h_energyRes.Fill(target.energy - reference.energy)
        self.h_energyResVeta.Fill(reference.eta, target.energy - reference.energy)

        self.h_etaRes.Fill(target.eta - reference.eta)
        self.h_phiRes.Fill(target.phi - reference.phi)
        self.h_drRes.Fill(np.sqrt((reference.phi-target.phi)**2+(reference.eta-target.eta)**2))
        if 'etalw' in target:
            self.h_etalwRes.Fill(target.etalw - reference.eta)
        if 'philw' in target:
            self.h_philwRes.Fill(target.philw - reference.phi)


class ResoHistos(BaseResoHistos):
    # @profile
    def __init__(self, name, root_file=None, debug=False):
        if not root_file:
            # self.h_ptRes = TH1F(
            #     name+'_ptRes', '3D Cluster Pt reso (GeV); (p_{T}^{L1} - p_{T}^{GEN})/p_{T}^{GEN}',
            #     100, -1, 1)
            # # self.h_energyRes = TH1F(name+'_energyRes', '3D Cluster Energy reso (GeV); E^{L1} - E^{GEN} [GeV]', 200, -100, 100)
            # self.h_ptResVeta = TH2F(
            #     name+'_ptResVeta', '3D Cluster Pt reso (GeV) vs eta; #eta^{GEN}; p_{T}^{L1} - p_{T}^{GEN} [GeV];',
            #     50, -3.5, 3.5, 20, -40, 40)
            # # self.h_energyResVenergy = TH2F(
            # #     name+'_energyResVenergy',
            # #     '3D Cluster E reso vs E; E^{GEN} [GeV]; (E^{L1} - E^{GEN})/E^{GEN};',
            # #     100, 0, 1000, 100, -1.5, 1.5)
            # # self.h_energyResVeta = TH2F(
            # #     name+'_energyResVeta',
            # #     '3D Cluster E reso (GeV) vs eta; #eta^{GEN}; (E^{L1} - E^{GEN})/E^{GEN};',
            # #     100, -3.5, 3.5, 100, -1.5, 1.5)
            # # self.h_energyResVnclu = TH2F(name+'_energyResVnclu', '3D Cluster E reso (GeV) vs # clusters; # 2D clus.; E^{L1} - E^{GEN} [GeV];', 50, 0, 50, 200, -100, 100)
            # self.h_ptResVpt = TH2F(
            #     name+'_ptResVpt',
            #     '3D Cluster Pt reso (GeV) vs pt (GeV); p_{T}^{GEN} [GeV]; p_{T}^{L1} - p_{T}^{GEN} [GeV];',
            #     50, 0, 100, 200, -40, 40)
            # self.h_ptResVnclu = TH2F(name+'_ptResVnclu', '3D Cluster Pt reso (GeV) vs # clusters; # 2D clus.; p_{T}^{L1} - p_{T}^{GEN} [GeV];', 50, 0, 50, 200, -40, 40)

            self.h_ptResp = TH1F(
                f'{name}_ptResp',
                '3D Cluster Pt resp.; p_{T}^{L1}/p_{T}^{GEN}',
                100, 0, 2)
            self.h_ptRespVpt = TH2F(
                f'{name}_ptRespVpt',
                '3D Cluster Pt resp. vs pt (GeV); p_{T}^{GEN} [GeV]; p_{T}^{L1}/p_{T}^{GEN};',
                50, 0, 100, 100, 0, 2)
            self.h_ptRespVeta = TH2F(
                f'{name}_ptRespVeta',
                '3D Cluster Pt resp. vs #eta; #eta^{GEN}; p_{T}^{L1}/p_{T}^{GEN};',
                20, -4, 4, 50, 0, 2)
            # self.h_ptRespVnclu = TH2F(
            #     name+'_ptRespVnclu',
            #     '3D Cluster Pt resp. vs # clus.; # 2D clust. ; p_{T}^{L1}/p_{T}^{GEN};',
            #     50, 0, 100, 100, 0, 2)
            # self.h_ptRespVetaVptL1 = ROOT.TH3F(
            #     name+'_ptRespVetaVptL1',
            #     '3D Cluster Pt resp. vs #eta and vs pT; #eta^{L1}; p_{T}^{L1} [GeV]; p_{T}^{L1}/p_{T}^{GEN};',
            #     30, 1, 4, 50, 0, 100, 100, 0, 3)

            # self.h_ptemResp = TH1F(
            #     name+'_ptemResp',
            #     '3D Cluster Pt resp.; p_{T}^{L1}/p_{T}^{GEN}',
            #     100, 0, 3)
            # self.h_ptemRespVpt = TH2F(
            #     name+'_ptemRespVpt',
            #     '3D Cluster Pt resp. vs pt (GeV); p_{T}^{GEN} [GeV]; p_{T}^{L1}/p_{T}^{GEN};',
            #     50, 0, 100, 100, 0, 3)

            # self.h_coreEnergyResVnclu = TH2F(name+'_coreEnergyResVnclu', '3D Cluster E reso (GeV) vs # clusters', 50, 0, 50, 200, -100, 100)
            # self.h_corePtResVnclu = TH2F(name+'_corePtResVnclu', '3D Cluster Pt reso (GeV) vs # clusters', 50, 0, 50, 200, -40, 40)
            #
            # self.h_coreEnergyRes = TH1F(name+'_coreEnergyRes', '3D Cluster Energy reso CORE (GeV)', 200, -100, 100)
            # self.h_corePtRes = TH1F(name+'_corePtRes', '3D Cluster Pt reso CORE (GeV)', 200, -40, 40)

            # self.h_centralEnergyRes = TH1F(name+'_centralEnergyRes', '3D Cluster Energy reso CENTRAL (GeV)', 200, -100, 100)
            self.h_etaRes = TH1F(
                f'{name}_etaRes',
                '3D Cluster eta reso; #eta^{L1}-#eta^{GEN}',
                100, -0.15, 0.15)
            self.h_phiRes = TH1F(
                f'{name}_phiRes',
                '3D Cluster phi reso; #phi^{L1}-#phi^{GEN}',
                100, -0.15, 0.15)
            self.h_drRes = TH1F(
                f'{name}_drRes',
                '3D Cluster DR reso; #DeltaR^{L1}-#DeltaR^{GEN}',
                100, 0, 0.1)
            # self.h_n010 = TH1F(
            #     name+'_n010',
            #     '# of 3D clus in 0.2 cone with pt>0.1GeV',
            #     10, 0, 10)
            # self.h_n025 = TH1F(
            #     name+'_n025',
            #     '# of 3D clus in 0.2 cone with pt>0.25GeV',
            #     10, 0, 10)

        BaseResoHistos.__init__(self, name, root_file)

    def fill(self, reference, target):
        fill_1Dhist(self.h_ptResp, target.pt/reference.pt)
        fill_2Dhist(self.h_ptRespVeta, reference.eta, target.pt/reference.pt)
        fill_2Dhist(self.h_ptRespVpt, reference.pt, target.pt/reference.pt)
        if 'caloeta' in reference.fields:
            fill_1Dhist(self.h_etaRes, target.eta - reference.caloeta)
            fill_1Dhist(self.h_phiRes, target.phi - reference.calophi)
            # self.h_drRes.Fill(np.sqrt((reference.exphi-target_phi)**2+(reference.exeta-target_eta)**2))

        # if 'n010' in target:
        #     self.h_n010.Fill(target_line.n010)
        # if 'n025' in target:
        #     self.h_n025.Fill(target_line.n025)


class Reso2DHistos(BaseHistos):
    def __init__(self, name, root_file=None, debug=False):
        if not root_file:
            # self.h_etaRes = TH1F(name+'_etaRes', 'Eta 2D cluster - GEN part', 100, -0.5, 0.5)
            # self.h_phiRes = TH1F(name+'_phiRes', 'Phi 2D cluster - GEN part', 100, -0.5, 0.5)
            # self.h_phiPRes = TH1F(name+'_phiPRes', 'Phi (+) 2D cluster - GEN part', 100, -0.5, 0.5)
            # self.h_phiMRes = TH1F(name+'_phiMRes', 'Phi (-) 2D cluster - GEN part', 100, -0.5, 0.5)
            self.h_xResVlayer = TH2F(f'{name}_xResVlayer', 'X resolution (cm) [(2D clus) - GEN]', 60, 0, 60, 100, -10, 10)
            self.h_yResVlayer = TH2F(f'{name}_yResVlayer', 'Y resolution (cm) [(2D clus) - GEN]', 60, 0, 60, 100, -10, 10)
            # self.h_DRRes = TH1F(name+'_DRRes', 'DR 2D cluster - GEN part', 100, -0.5, 0.5)

        BaseHistos.__init__(self, name, root_file, debug)

    def fill(self, reference, target):
        # rnp.fill_hist(self.h_etaRes, reference.eta-target.eta)
        #
        # rnp.fill_hist(self.h_phiRes, reference.phi-target.phi)
        # if reference.pdgid < 0:
        #     rnp.fill_hist(self.h_phiMRes, reference.phi-target.phi)
        # elif reference.pdgid > 0:
        #     rnp.fill_hist(self.h_phiPRes, reference.phi-target.phi)

        # rnp.fill_hist(self.h_DRRes, np.sqrt((reference.phi-target.phi)**2+(reference.eta-target.eta)**2))
        if reference.reachedEE == 2:
            if 'x' in target.columns:
                target['xres'] = reference.posx[target.layer-1]-target.x
                # print target[['layer', 'xres']]
                rnp.fill_hist(self.h_xResVlayer, target[['layer', 'xres']])
            if 'y' in target.columns:
                target['yres'] = reference.posy[target.layer-1]-target.y
                # print target[['layer', 'yres']]
                rnp.fill_hist(self.h_yResVlayer, target[['layer', 'yres']])
            # print target[['layer', 'xres', 'yres']]


class GeomHistos(BaseHistos):
    def __init__(self, name, root_file=None, debug=False):
        if not root_file:
            self.h_maxNNDistVlayer = TH2F(f'{name}_maxNNDistVlayer', 'Max dist between NN vs layer', 60, 0, 60, 100, 0, 10)
            self.h_minNNDistVlayer = TH2F(f'{name}_minNNDistVlayer', 'Max dist between NN vs layer', 60, 0, 60, 100, 0, 10)

            self.h_nTCsPerLayer = TH1F(f'{name}_nTCsPerLayer', '# of Trigger Cells per layer', 60, 0, 60)
            self.h_radiusVlayer = TH2F(f'{name}_radiusVlayer', '# of cells radius vs layer', 60, 0, 60, 200, 0, 200)
        BaseHistos.__init__(self, name, root_file, debug)

    def fill(self, tcs):
        if True:
            ee_tcs = tcs[tcs.subdet == 3]
            for index, tc_geom in ee_tcs.iterrows():
                self.h_maxNNDistVlayer.Fill(tc_geom.layer, np.max(tc_geom.neighbor_distance))
                self.h_minNNDistVlayer.Fill(tc_geom.layer, np.min(tc_geom.neighbor_distance))

        rnp.fill_hist(self.h_nTCsPerLayer, tcs[tcs.subdet == 3].layer)
        rnp.fill_hist(self.h_radiusVlayer, tcs[['layer', 'radius']])


class DensityHistos(BaseHistos):
    def __init__(self, name, root_file=None, debug=False):
        if not root_file:
            self.h_eDensityVlayer = TH2F(f'{name}_eDensityVlayer', 'E (GeV) Density per layer', 60, 0, 60, 600, 0, 30)
            self.h_nTCDensityVlayer = TH2F(f'{name}_nTCDensityVlayer', '# TC Density per layer', 60, 0, 60, 20, 0, 20)
        elif 'v7' in root_file.GetName() and 'NuGun' not in root_file.GetName():
            print('v7 hack')
            self.h_eDensityVlayer = root_file.Get(f'{name}eDensityVlayer')
            self.h_nTCDensityVlayer = root_file.Get(f'{name}nTCDensityVlayer')
        BaseHistos.__init__(self, name, root_file, debug)

    def fill(self, layer, energy, nTCs):
        self.h_eDensityVlayer.Fill(layer, energy)
        self.h_nTCDensityVlayer.Fill(layer, nTCs)


# for convenience we define some sets
class HistoSetClusters:
    def __init__(self, name, root_file=None, debug=False):
        self.htc = TCHistos(f'h_tc_{name}', root_file, debug)
        self.hcl2d = ClusterHistos(f'h_cl2d_{name}', root_file, debug)
        self.hcl3d = Cluster3DHistos(f'h_cl3d_{name}', root_file, debug)
        # if not root_file:
        #     self.htc.annotateTitles(name)
        #     self.hcl2d.annotateTitles(name)
        #     self.hcl3d.annotateTitles(name)

    def fill(self, tcs, cl2ds, cl3ds):
        self.htc.fill(tcs)
        self.hcl2d.fill(cl2ds)
        self.hcl3d.fill(cl3ds)


class HistoSetReso:
    def __init__(self, name, root_file=None, debug=False):
        self.hreso = ResoHistos(f'h_reso_{name}', root_file, debug)
        self.hresoCone = None
        self.hreso2D = None
        # self.hresoCone = ResoHistos('h_resoCone_'+name, root_file)
        # self.hreso2D = Reso2DHistos('h_reso2D_'+name, root_file)
        # if not root_file:
        #     self.hreso.annotateTitles(name)
        #     self.hresoCone.annotateTitles(name)
        #     self.hreso2D.annotateTitles(name)


class HistoEff:
    def __init__(self, passed, total, rebin=None, debug=False):
        # print dir(total)
        for histo in [a for a in dir(total) if a.startswith('h_')]:
            if debug:
                print(histo)
            hist_total = getattr(total, histo)
            hist_passed = getattr(passed, histo)
            if rebin is None:
                setattr(self, histo, ROOT.TEfficiency(hist_passed, hist_total))
            else:
                setattr(self, histo, ROOT.TEfficiency(hist_passed.Rebin(rebin,
                                                                        f'{hist_passed.GetName()}_rebin{rebin}'),
                                                      hist_total.Rebin(rebin,
                                                                       f'{hist_total.GetName()}_rebin{rebin}')))
            # getattr(self, histo).Sumw2()


class HistoSetEff:
    def __init__(self, name, root_file=None, pt_bins=None, debug=False):
        self.name = name
        self.h_num = GenParticleHistos(f'h_effNum_{name}', root_file, pt_bins=pt_bins, debug=debug)
        self.h_den = GenParticleHistos(f'h_effDen_{name}', root_file, pt_bins=pt_bins, debug=debug)
        self.h_eff = None
        self.h_ton = None

        if root_file:
            self.computeEff(debug=debug)

    def fillNum(self, particles):
        self.h_num.fill(particles)

    def fillDen(self, particles):
        self.h_den.fill(particles)

    def computeEff(self, rebin=None, debug=False):
        # print "Computing eff"
        if self.h_eff is None or rebin is not None:
            self.h_eff = HistoEff(passed=self.h_num, total=self.h_den, rebin=rebin, debug=debug)

    def computeTurnOn(self, denominator, debug=False):
        self.h_ton = HistoEff(passed=self.h_num, total=denominator, debug=debug)


class TrackResoHistos(BaseResoHistos):
    def __init__(self, name, root_file=None, debug=False):
        if not root_file:
            self.h_ptResp = TH1F(
                f'{name}_ptResp',
                'Track Pt resp.; p_{T}^{L1}/p_{T}^{GEN}',
                100, 0, 3)
            self.h_ptRespVpt = TH2F(
                f'{name}_ptRespVpt',
                'Track Pt resp. vs pt (GeV); p_{T}^{GEN} [GeV]; p_{T}^{L1}/p_{T}^{GEN};',
                50, 0, 100, 100, 0, 3)
            self.h_ptRespVeta = TH2F(
                f'{name}_ptRespVeta',
                'Track Pt resp. vs #eta; #eta^{GEN}; p_{T}^{L1}/p_{T}^{GEN};',
                50, -4, 4, 100, 0, 3)
            self.h_etaRes = TH1F(
                f'{name}_etaRes',
                'Track eta reso',
                100, -0.4, 0.4)
            self.h_phiRes = TH1F(
                f'{name}_phiRes',
                'Track phi reso',
                100, -0.4, 0.4)
            self.h_drRes = TH1F(
                f'{name}_drRes',
                'Track DR reso',
                100, 0, 0.4)
            self.h_nMatch = TH1F(
                f'{name}_nMatch',
                '# matches',
                100, 0, 100)

            # self.h_pt2stResVpt = TH2F(name+'_pt2stResVpt', 'EG Pt 2stubs reso. vs pt (GeV); p_{T}^{GEN} [GeV]; p_{T}^{L1}-p_{T}^{GEN} [GeV];',
            #                                50, 0, 100, 100, -20, 20)
            #
            # self.h_pt2stResp = TH1F(name+'_pt2stResp', 'Track Pt resp.; p_{T}^{L1}/p_{T}^{GEN}',
            #                              100, 0, 3)
            # self.h_pt2stRespVpt = TH2F(name+'_pt2stRespVpt', 'Track Pt resp. vs pt (GeV); p_{T}^{GEN} [GeV]; p_{T}^{L1}/p_{T}^{GEN};',
            #                                 50, 0, 100, 100, 0, 3)
            # self.h_pt2stRespVeta = TH2F(name+'_pt2stRespVeta', 'Track Pt resp. vs #eta; #eta^{GEN}; p_{T}^{L1}/p_{T}^{GEN};',
            #                                  50, -4, 4, 100, 0, 3)

        BaseResoHistos.__init__(self, name, root_file, debug)

    def fill(self, reference, target):
        fill_1Dhist(self.h_ptResp, target.pt/reference.pt)
        fill_2Dhist(self.h_ptRespVeta, reference.eta, target.pt/reference.pt)
        fill_2Dhist(self.h_ptRespVpt, reference.pt, target.pt/reference.pt)

        # self.h_pt2stResVpt.Fill(reference.pt, target.pt2stubs-reference.pt)
        # self.h_pt2stResp.Fill(target.pt2stubs/reference.pt)
        # self.h_pt2stRespVeta.Fill(reference.eta, target.pt2stubs/reference.pt)
        # self.h_pt2stRespVpt.Fill(reference.pt, target.pt2stubs/reference.pt)

        fill_1Dhist(self.h_etaRes, (target.eta - reference.eta))
        fill_1Dhist(self.h_phiRes, (target.phi - reference.phi))
        fill_1Dhist(self.h_drRes, np.sqrt((reference.phi-target.phi)**2+(reference.eta-target.eta)**2))

    def fill_nMatch(self, n_matches):
        self.h_nMatch.Fill(n_matches)


class JetResoHistos(BaseResoHistos):
    def __init__(self, name, root_file=None, debug=False):
        if not root_file:
            self.h_ptResp = TH1F(
                f'{name}_ptResp',
                'Track Pt resp.; p_{T}^{L1}/p_{T}^{GEN}',
                100, 0, 3)
            self.h_ptRespVpt = TH2F(
                f'{name}_ptRespVpt',
                'Track Pt resp. vs pt (GeV); p_{T}^{GEN} [GeV]; p_{T}^{L1}/p_{T}^{GEN};',
                50, 0, 100, 100, 0, 3)
            self.h_ptRespVeta = TH2F(
                f'{name}_ptRespVeta',
                'Track Pt resp. vs #eta; #eta^{GEN}; p_{T}^{L1}/p_{T}^{GEN};',
                50, -4, 4, 100, 0, 3)
            self.h_etaRes = TH1F(
                f'{name}_etaRes',
                'Track eta reso',
                100, -0.4, 0.4)
            self.h_phiRes = TH1F(
                f'{name}_phiRes',
                'Track phi reso',
                100, -0.4, 0.4)
            self.h_drRes = TH1F(
                f'{name}_drRes',
                'Track DR reso',
                100, 0, 0.4)
            self.h_nMatch = TH1F(
                f'{name}_nMatch',
                '# matches',
                100, 0, 100)

        BaseResoHistos.__init__(self, name, root_file, debug)

    def fill(self, reference, target):
        fill_1Dhist(self.h_ptResp, target.pt/reference.pt)
        fill_2Dhist(self.h_ptRespVeta, reference.eta, target.pt/reference.pt)
        fill_2Dhist(self.h_ptRespVpt, reference.pt, target.pt/reference.pt)
        fill_1Dhist(self.h_etaRes, (target.eta - reference.eta))
        fill_1Dhist(self.h_phiRes, (target.phi - reference.phi))
        fill_1Dhist(self.h_drRes, np.sqrt((reference.phi-target.phi)**2+(reference.eta-target.eta)**2))

    def fill_nMatch(self, n_matches):
        self.h_nMatch.Fill(n_matches)


class DecTkResoHistos(BaseResoHistos):
    def __init__(self, name, root_file=None, debug=False):
        if not root_file:
            self.h_ptResVpt = TH2F(
                f'{name}_ptResVpt',
                'Track Pt reso. vs pt (GeV); p_{T}^{GEN} [GeV]; p_{T}^{L1}-p_{T}^{GEN} [GeV];',
                50, 0, 100, 100, -20, 20)
            self.h_ptResp = TH1F(
                f'{name}_ptResp',
                'Track Pt resp.; p_{T}^{L1}/p_{T}^{GEN}',
                100, 0, 3)
            self.h_ptRespVpt = TH2F(
                f'{name}_ptRespVpt',
                'Track Pt resp. vs pt (GeV); p_{T}^{GEN} [GeV]; p_{T}^{L1}/p_{T}^{GEN};',
                50, 0, 100, 100, 0, 3)
            self.h_ptRespVeta = TH2F(
                f'{name}_ptRespVeta',
                'Track Pt resp. vs #eta; #eta^{GEN}; p_{T}^{L1}/p_{T}^{GEN};',
                50, -4, 4, 100, 0, 3)
            self.h_etaRes = TH1F(
                f'{name}_etaRes',
                'Track eta reso',
                100, -0.15, 0.15)
            self.h_etaResVabseta = TH2F(
                f'{name}_etaResVabseta',
                '#eta_{@vtx} reso; |#eta^{GEN}|; #eta_{@vtx}^{L1} vs #eta_{@vtx}^{GEN}',
                50, 0, 2.5,
                100, -0.1, 0.1)
            self.h_etaResVeta = TH2F(
                f'{name}_etaResVeta',
                '#eta_{@vtx} reso; #eta^{GEN}; #eta_{@vtx}^{L1} vs #eta_{@vtx}^{GEN}',
                200, -2.5, 2.5,
                100, -0.1, 0.1)
            self.h_phiRes = TH1F(
                f'{name}_phiRes',
                'Track phi reso',
                100, -0.4, 0.4)
            self.h_caloEtaRes = TH1F(
                f'{name}_caloEtaRes',
                '#eta_{@calo} reso; #eta_{@calo}^{L1} vs #eta_{@calo}^{GEN}',
                100, -0.15, 0.15)
            self.h_caloEtaResVabseta = TH2F(
                f'{name}_caloEtaResVabseta',
                '#eta_{@calo} reso; |#eta^{GEN}|; #eta_{@calo}^{L1} vs #eta_{@calo}^{GEN}',
                50, 0, 2.5,
                100, -0.1, 0.1)
            self.h_caloEtaResVeta = TH2F(
                f'{name}_caloEtaResVeta',
                '#eta_{@calo} reso; #eta^{GEN}; #eta_{@calo}^{L1} vs #eta_{@calo}^{GEN}',
                200, -2.5, 2.5,
                100, -0.1, 0.1)
            self.h_caloPhiRes = TH1F(
                f'{name}_caloPhiRes',
                '#phi_{@calo} reso; #phi_{@calo}^{L1} vs #phi_{@calo}^{GEN}',
                100, -0.4, 0.4)
            self.h_caloPhiResVabseta = TH2F(
                f'{name}_caloPhiResVabseta',
                '#phi_{@calo} reso; |#eta^{GEN}|; #phi_{@calo}^{L1} vs #phi_{@calo}^{GEN}',
                50, 0, 3,
                100, -0.4, 0.4)
            self.h_dzRes = TH1F(
                f'{name}_dzRes',
                '#DeltaZ_{0} res; #DeltaZ_{0}^{L1}-#DeltaZ_{0}^{GEN}',
                100, -10, 10)

            # self.h_caloPhiResVeta = TH2F(
            #     name+'_caloPhiResVabseta',
            #     '#phi_{@calo} reso; #eta^{GEN}; #phi_{@calo}^{L1} vs #phi_{@calo}^{GEN}',
            #     50, 0, 3,
            #     100, -0.4, 0.4)
            self.h_nMatch = TH1F(
                f'{name}_nMatch',
                '# matches',
                100, 0, 100)

            # self.h_pt2stResVpt = TH2F(name+'_pt2stResVpt', 'EG Pt 2stubs reso. vs pt (GeV); p_{T}^{GEN} [GeV]; p_{T}^{L1}-p_{T}^{GEN} [GeV];',
            #                                50, 0, 100, 100, -20, 20)
            #
            # self.h_pt2stResp = TH1F(name+'_pt2stResp', 'Track Pt resp.; p_{T}^{L1}/p_{T}^{GEN}',
            #                              100, 0, 3)
            # self.h_pt2stRespVpt = TH2F(name+'_pt2stRespVpt', 'Track Pt resp. vs pt (GeV); p_{T}^{GEN} [GeV]; p_{T}^{L1}/p_{T}^{GEN};',
            #                                 50, 0, 100, 100, 0, 3)
            # self.h_pt2stRespVeta = TH2F(name+'_pt2stRespVeta', 'Track Pt resp. vs #eta; #eta^{GEN}; p_{T}^{L1}/p_{T}^{GEN};',
            #                                  50, -4, 4, 100, 0, 3)

        BaseResoHistos.__init__(self, name, root_file, debug)

    def fill(self, reference, target):
        # target_pt, target_eta, target_phi = \
        #     target[['pt', 'eta', 'phi']].values[0]
        # reference_pt, reference_eta, reference_phi = \
        #     reference[['pt', 'eta', 'phi']].values
        target_line = target.iloc[0]

        target_pt = target_line.pt
        target_eta = target_line.eta
        target_phi = target_line.phi
        reference_pt = reference.pt
        reference_eta = reference.eta
        reference_phi = reference.phi

        self.h_ptResVpt.Fill(reference_pt, target_pt-reference_pt)
        self.h_ptResp.Fill(target_pt/reference_pt)
        self.h_ptRespVeta.Fill(reference_eta, target_pt/reference_pt)
        self.h_ptRespVpt.Fill(reference_pt, target_pt/reference_pt)

        # self.h_pt2stResVpt.Fill(reference.pt, target.pt2stubs-reference.pt)
        # self.h_pt2stResp.Fill(target.pt2stubs/reference.pt)
        # self.h_pt2stRespVeta.Fill(reference.eta, target.pt2stubs/reference.pt)
        # self.h_pt2stRespVpt.Fill(reference.pt, target.pt2stubs/reference.pt)

        self.h_etaRes.Fill(target_eta - reference_eta)
        self.h_etaResVabseta.Fill(reference.abseta, target_eta - reference_eta)
        self.h_etaResVeta.Fill(reference.eta, target_eta - reference_eta)

        self.h_phiRes.Fill(target_phi - reference_phi)
        self.h_caloEtaRes.Fill(target_line.caloeta - reference.exeta)
        self.h_caloPhiRes.Fill(target_line.calophi - reference.exphi)
        self.h_caloEtaResVabseta.Fill(reference.abseta, target_line.caloeta - reference.exeta)
        self.h_caloPhiResVabseta.Fill(reference.abseta, target_line.calophi - reference.exphi)
        self.h_caloEtaResVeta.Fill(reference_eta, target_line.caloeta - reference.exeta)
        # self.h_caloPhiResVeta.Fill(reference_eta, target_line.calophi - reference.exphi)
        self.h_dzRes.Fill(target_line.z0 - reference.ovz)

    def fill_nMatch(self, n_matches):
        self.h_nMatch.Fill(n_matches)


class EGResoHistos(BaseResoHistos):
    def __init__(self, name, root_file=None, debug=False):
        if not root_file:

            self.h_ptResVpt = TH2F(
                f'{name}_ptResVpt',
                'EG Pt reso. vs pt (GeV); p_{T}^{GEN} [GeV]; p_{T}^{L1}-p_{T}^{GEN} [GeV];',
                50, 0, 100,
                100, -10, 10)
            self.h_ptRes = TH1F(
                f'{name}_ptRes',
                'EG Pt res.; (p_{T}^{L1}-p_{T}^{GEN})/p_{T}^{GEN}',
                100, -1, 1)
            self.h_ptResp = TH1F(
                f'{name}_ptResp',
                'EG Pt resp.; p_{T}^{L1}/p_{T}^{GEN}',
                100, 0, 3)
            self.h_ptRespVpt = TH2F(
                f'{name}_ptRespVpt',
                'EG Pt resp. vs pt (GeV); p_{T}^{GEN} [GeV]; p_{T}^{L1}/p_{T}^{GEN};',
                50, 0, 100,
                100, 0, 3)
            self.h_ptRespVeta = TH2F(
                f'{name}_ptRespVeta',
                'EG Pt resp. vs #eta; #eta^{GEN}; p_{T}^{L1}/p_{T}^{GEN};',
                50, -4, 4,
                100, 0, 3)

            self.h_etaRes = TH1F(
                f'{name}_etaRes',
                'EG eta reso; #eta^{L1}-#eta^{GEN}',
                100, -0.1, 0.1)
            self.h_phiRes = TH1F(
                f'{name}_phiRes',
                'EG phi reso; #phi^{L1}-#phi^{GEN}',
                100, -0.1, 0.1)

            self.h_exetaRes = TH1F(
                f'{name}_exetaRes',
                'EG eta reso; #eta^{L1}-#eta^{GEN}_{calo}',
                100, -0.1, 0.1)
            self.h_exphiRes = TH1F(
                f'{name}_exphiRes',
                'EG phi reso; #phi^{L1}-#phi^{GEN}_{calo}',
                100, -0.1, 0.1)

            self.h_dzRes = TH1F(
                f'{name}_dzRes',
                '#DeltaZ_{0} res; #DeltaZ_{0}^{L1}-#DeltaZ_{0}^{GEN}',
                100, -10, 10)

        BaseResoHistos.__init__(self, name, root_file, debug)

    def fill(self, reference, target):
        # FIXME: weights

        fill_1Dhist(self.h_ptRes, (target.pt-reference.pt)/reference.pt)
        fill_2Dhist(self.h_ptResVpt, reference.pt, target.pt-reference.pt)
        fill_1Dhist(self.h_ptResp, target.pt/reference.pt)
        fill_2Dhist(self.h_ptRespVeta, reference.eta, target.pt/reference.pt)
        fill_2Dhist(self.h_ptRespVpt, reference.pt, target.pt/reference.pt)
        fill_1Dhist(self.h_etaRes, target.eta - reference.eta)
        fill_1Dhist(self.h_phiRes, target.phi - reference.phi)
        fill_1Dhist(self.h_exetaRes, target.eta - reference.caloeta)
        fill_1Dhist(self.h_exphiRes, target.phi - reference.calophi)

        # if 'tkZ0' in target.columns:
        #     self.h_dzRes.Fill(target_line.tkZ0 - reference.ovz)


class ClusterConeHistos(BaseHistos):
    def __init__(self, name, root_file=None, debug=False):
        if not root_file:
            self.h_ptRel = TH1F(f'{name}_ptRel',
                                     'Pt best/Pt other; p_{T}^{best}/p_{T}^{other}', 100, 0, 5)
            self.h_ptRelVpt = TH2F(f'{name}_ptRelVpt', 'Pt best/Pt other vs pt (GeV); p_{T}^{best} [GeV]; p_{T}^{best}/p_{T}^{other};', 50, 0, 100, 100, 0, 5)
            self.h_deltaEta = TH1F(f'{name}_deltaEta', '#Delta eta; #eta^{best}-#eta^{other}', 100, -0.4, 0.4)
            self.h_deltaPhi = TH1F(f'{name}_deltaPhi', '#Delta phi; #phi^{best}-#phi^{other}', 100, -0.4, 0.4)
            self.h_deltaPhiVq = TH2F(f'{name}_deltaPhiVq', '#Delta phi; #phi^{best}-#phi^{other}; GEN charge;', 100, -0.4, 0.4, 3, -1, 2)

            self.h_deltaR = TH1F(f'{name}_deltaR', '#Delta R (best-other); #Delta R (best, other)', 100, 0, 0.4)
            self.h_n = TH1F(f'{name}_n', '# other clusters in cone; # others', 20, 0, 20)
        BaseHistos.__init__(self, name, root_file, debug)

    def fill(self, reference, target, charge):
        self.h_ptRel.Fill(target.pt/reference.pt)
        self.h_ptRelVpt.Fill(reference.pt, target.pt/reference.pt)
        self.h_deltaEta.Fill(target.eta - reference.eta)
        self.h_deltaPhi.Fill(target.phi - reference.phi)
        # self.h_deltaPhi.Fill(target.phi - reference.phi)
        self.h_deltaPhiVq.Fill(target.phi - reference.phi, charge)
        self.h_deltaR.Fill(np.sqrt((reference.phi-target.phi)**2+(reference.eta-target.eta)**2))

    def fill_n(self, num):
        self.h_n.Fill(num)


class IsoTuples(BaseTuples):
    def __init__(self, name, root_file=None, debug=False):
        BaseTuples.__init__(
            self, 'iso', 'pid_gen:e_gen:pt_gen:eta_gen:z0_gen:e:pt:eta:tkIso:pfIso:tkIsoPV:pfIsoPV',
            name, root_file, debug)

    def fill(self, reference, target):
        values_fill = []

        if reference is not None:
            values_fill.append(reference.pid)
            values_fill.append(reference.energy)
            values_fill.append(reference.pt)
            values_fill.append(reference.eta)
            values_fill.append(reference.ovz)
        else:
            values_fill.extend([-1]*5)

        values_fill.append(target.energy)
        values_fill.append(target.pt)
        values_fill.append(target.eta)
        values_fill.append(target.tkIso)
        values_fill.append(target.pfIso)
        if 'tkIsoPV' in target.keys():
            values_fill.append(target.tkIsoPV)
            values_fill.append(target.pfIsoPV)
        else:
            values_fill.append(-1)
            values_fill.append(-1)

        self.t_values.Fill(array('f', values_fill))


class ResoTuples(BaseUpTuples):
    def __init__(self, name, root_file=None, debug=False):
        BaseUpTuples.__init__(
            self, 'ResoData', name, root_file, debug)

    def fill(self, reference, target):
        # print(self.t_name)
        # print(target.fields)
        target_vars = [
            'pt',
            'eta',
            'phi',
            'energy',
            'hwQual']
        rference_vars = [
            'pt',
            'eta',
            'phi',
            'energy']
        tree_data = {}
        for var in target_vars:
            tree_data[var] = ak.flatten(ak.drop_none(target[var]))
        for var in rference_vars:
            tree_data[f'gen_{var}'] = ak.flatten(ak.drop_none(reference[var]))
        # print(reference.fields)
        # tree_data[f'gen_dz'] = ak.flatten(ak.drop_none(np.abs(reference.ovz-target.tkZ0)))

        BaseUpTuples.fill(self, tree_data)


class CalibrationHistos(BaseTuples):
    def __init__(self, name, root_file=None, debug=False):
        BaseTuples.__init__(
            self, 'calib',
            'e1:e3:e5:e7:e9:e11:e13:e15:e17:e19:e21:e23:e25:e27:Egen:eta:pt:ptgen',
            name, root_file, debug)

    def fill(self, reference, target):
        # cluster_data = []
        # self.data.append(target.iloc[0]['layer_energy'])
        # self.reference.append(reference.energy)
        values_fill = []
        values_fill.extend(target.iloc[0]['layer_energy'])
        values_fill.append(reference.energy)
        values_fill.append(target.eta)
        values_fill.append(target.pt)
        values_fill.append(reference.pt)
        self.t_values.Fill(array('f', values_fill))



# for convenience we define some sets
class HistoSetOccupancy:
    def __init__(self, name, root_file=None, debug=False):
        self.htc = TCHistos(f'h_tc_{name}', root_file, debug)
        self.hcl2d = ClusterHistos(f'h_cl2d_{name}', root_file, debug)
        self.hcl3d = Cluster3DHistos(f'h_cl3d_{name}', root_file, debug)
        # if not root_file:
        #     self.htc.annotateTitles(name)
        #     self.hcl2d.annotateTitles(name)
        #     self.hcl3d.annotateTitles(name)

    def fill(self, tcs, cl2ds, cl3ds):
        self.htc.fill(tcs)
        self.hcl2d.fill(cl2ds)
        self.hcl3d.fill(cl3ds)






class TCClusterMatchHistos(BaseHistos):
    def __init__(self, name, root_file=None, debug=False):
        if not root_file:
            self.h_dEtaVdPhi = TH2F(
                f'{name}_dEtaVdPhi',
                '#Delta#eta vs #Delta#phi; #Delta#phi [rad]; #Delta#eta;',
                100, -0.1, 0.1, 100, -0.1, 0.1)
            self.h_dEtaRMSVenergy = ROOT.TProfile(
                f'{name}_dEtaRMSVenergy',
                'RMS(#Delta#eta) vs energy; E [GeV]; RMS(#Delta#eta);',
                100, 0, 1000)
            self.h_dEtaRMSVpt = ROOT.TProfile(
                f'{name}_dEtaRMSVpt',
                'RMS(#Delta#eta) vs pt; p_{T} [GeV]; RMS(#Delta#eta);',
                100, 0, 100)
            self.h_dPhiRMSVenergy = ROOT.TProfile(
                f'{name}_dPhiRMSVenergy',
                'RMS(#Delta#phi) vs energy; E [GeV]; RMS(#Delta#phi);',
                100, 0, 1000)
            self.h_dPhiRMSVpt = ROOT.TProfile(
                f'{name}_dPhiRMSVpt',
                'RMS(#Delta#phi) vs pt; p_{T} [GeV]; RMS(#Delta#phi);',
                100, 0, 100)
            self.h_dRhoRMSVenergy = ROOT.TProfile(
                f'{name}_dRhoRMSVenergy',
                'RMS(#Delta#rho) vs energy; E [GeV]; RMS(#Delta#rho);',
                100, 0, 1000)
            self.h_dRhoRMSVpt = ROOT.TProfile(
                f'{name}_dRhoRMSVpt',
                'RMS(#Delta#rho) vs pt; p_{T} [GeV]; RMS(#Delta#rho);',
                100, 0, 100)
            self.h_dRho = TH1F(
                f'{name}_dRho',
                '#Delta#rho; #Delta#rho;',
                100, 0, 0.1)
            self.h_dRho2 = TH1F(
                f'{name}_dRho2',
                '#Delta#rho (E fraction weighted); #Delta#rho;',
                100, 0, 0.1)

            self.h_dRhoVlayer = TH2F(
                f'{name}_dRhoVlayer',
                '#Delta#rho; layer #; #Delta#rho;',
                60, 0, 60, 100, 0, 0.1)
            self.h_dRhoVabseta = TH2F(
                f'{name}_dRhoVabseta',
                '#Delta#rho; |#eta|; #Delta#rho;',
                100, 1.4, 3.1, 100, 0, 0.1)
            # self.h_dRhoVfbrem = TH2F(name+'_dRhoVfbrem',
            #                         '#Delta#rho vs f_{brem}; f_{brem}; #Delta#rho;',
            #                         100, 0, 1, 100, 0, 0.1)
            self.h_dtVlayer = TH2F(
                f'{name}_dtVlayer',
                '#Deltat vs layer; layer #; #Deltat;',
                60, 0, 60, 100, -0.05, 0.05)
            self.h_duVlayer = TH2F(
                f'{name}_duVlayer',
                '#Delta#rho; layer #; #Deltau;',
                60, 0, 60, 100, -0.05, 0.05)

            self.h_dtVlayer2 = TH2F(
                f'{name}_dtVlayer2',
                '#Deltat vs layer; layer #; #Deltat;',
                60, 0, 60, 100, -0.05, 0.05)
            self.h_duVlayer2 = TH2F(
                f'{name}_duVlayer2',
                '#Delta#rho; layer #; #Deltau;',
                60, 0, 60, 100, -0.05, 0.05)

            self.h_dtVdu = TH2F(
                f'{name}_dtVdu',
                '#Deltat vs #Deltau; #Deltat [cm]; #Deltau [cm];',
                100, -0.05, 0.05, 100, -0.05, 0.05)
            self.h_dtVdu2 = TH2F(
                f'{name}_dtVdu2',
                '#Deltat vs #Deltau (E fract. weighted); #Deltat [cm]; #Deltau [cm];',
                100, -0.05, 0.05, 100, -0.05, 0.05)
            # self.h_fbremVabseta = TH2F(name+'_fbremVabseta',
            #                         'f_{brem} vs |#eta|; |#eta|; f_{brem};',
            #                         100, 1.4, 3.1, 100, 0, 1)

        BaseHistos.__init__(self, name, root_file, debug)

    def fill(self, tcs, cluster):
        rnp.fill_hist(self.h_dEtaVdPhi, tcs[['delta_phi', 'delta_eta']])
        # print tcs.dr
        # print tcs.delta_eta.std(), tcs.delta_phi.std(), tcs.dr.std()

        self.h_dEtaRMSVenergy.Fill(cluster.energy, tcs.delta_eta.std())
        self.h_dEtaRMSVpt.Fill(cluster.pt, tcs.delta_eta.std())
        self.h_dPhiRMSVenergy.Fill(cluster.energy, tcs.delta_phi.std())
        self.h_dPhiRMSVpt.Fill(cluster.pt, tcs.delta_phi.std())
        self.h_dRhoRMSVenergy.Fill(cluster.energy, tcs.dr.std())
        self.h_dRhoRMSVpt.Fill(cluster.pt, tcs.dr.std())

        rnp.fill_hist(self.h_dRho, tcs.dr)
        rnp.fill_hist(self.h_dRho2, tcs.dr, tcs.ef)
        rnp.fill_hist(self.h_dRhoVlayer, tcs[['layer', 'dr']])
        rnp.fill_hist(self.h_dtVlayer2, tcs[['layer', 'dt']], tcs['ef'])
        rnp.fill_hist(self.h_duVlayer2, tcs[['layer', 'du']], tcs['ef'])

        rnp.fill_hist(self.h_dtVlayer, tcs[['layer', 'dt']])
        rnp.fill_hist(self.h_duVlayer, tcs[['layer', 'du']])

        rnp.fill_hist(self.h_dRhoVabseta, tcs[['abseta_cl', 'dr']])
        # rnp.fill_hist(self.h_dRhoVfbrem, tcs[['fbrem_cl', 'dr']])
        rnp.fill_hist(self.h_dtVdu, tcs[['dt', 'du']])
        rnp.fill_hist(self.h_dtVdu2, tcs[['dt', 'du']], tcs['ef'])
        # self.h_fbremVabseta.Fill(cluster.abseta, cluster.fbrem)



class QuantizationHistos(BaseHistos):
    def __init__(self, name, features=None, root_file=None, debug=False):
        if not root_file:
            self.features = features
            self.h_features = TH2F_category(
                f'{name}_features',
                'features; feature; value',
                 self.features,
                 1000, -1000, 1000)
            self.h_featuresLog2 = TH2F_category(
                f'{name}_featuresLog2',
                'featuresLog2; features; log_{2}(value)',
                 self.features,
                 64, -32, 32)
            # for bin,ft in enumerate(features):
            #     self.h_features.GetXaxis().SetBinLabel(bin+1, ft)
            #     self.h_featuresLog2.GetXaxis().SetBinLabel(bin+1, ft)

        BaseHistos.__init__(self, name, root_file, debug)

    def fill(self, df):
        fill = df
        # print(df.fields)
        for bin,ft in enumerate(self.features):
            fill[f'{ft}_bin'] = [ft]
            fill[f'{ft}_log2'] = np.log2(fill[ft])

            fill_2Dhist(self.h_features, fill[f'{ft}_bin'], fill[ft])
            fill_2Dhist(self.h_featuresLog2, fill[f'{ft}_bin'], fill[f'{ft}_log2'])


class DiObjMassHistos(BaseHistos):
    def __init__(self, name, root_file=None, debug=False):
        if not root_file:
            self.h_mass = TH1F(f'{name}_mass', 'mass (GeV); M(ll) [GeV]', 400, 0, 8000)
        BaseHistos.__init__(self, name, root_file, debug)

    def fill(self, obj_pairs):
        weight = None
        if 'weight' in obj_pairs.fields:
            weight = obj_pairs.weight
        objs_sum = obj_pairs.leg0+obj_pairs.leg1
        # fill_1Dhist(
        #     hist=self.h_mass,
        #     array=objs_sum[ak.count((objs_sum).pt, axis=1) > 0][:, 0].mass,
        #     weights=weight)
        self.h_mass.fill(objs_sum[ak.count((objs_sum).pt, axis=1) > 0][:, 0].mass)




# if __name__ == "__main__":
#     import sys
#     def createHisto(Class):
#         return Class(name='pippo_{}'.format(Class))
#
#     @profile
#     def createAll():
#         histos = []
#         histos.append(createHisto(Reso2DHistos))
#         # print sys.getsizeof(createHisto(Reso2DHistos))
#         histos.append(createHisto(GenParticleHistos))
#         histos.append(createHisto(RateHistos))
#         histos.append(createHisto(TCHistos))
#         histos.append(createHisto(ClusterHistos))
#         createHisto(Cluster3DHistos)
#         createHisto(TriggerTowerHistos)
#         createHisto(TriggerTowerResoHistos)
#         createHisto(ResoHistos)
#
#         return histos
#
#
#
#     createAll()

# 8. boost_hist.py

In [17]:
import hist.dask as dah
import hist
import awkward as ak

def TH1F(name, title, nbins, bin_low, bin_high):
    b_axis_name = 'X'
    title_split = title.split(';')
    if len(title_split) > 1:
        b_axis_name = title_split[1]
    b_name = title_split[0]
    b_label = name
        
    return hist.dask.Hist(
         hist.axis.Regular(bins=nbins, start=bin_low, stop=bin_high, name=b_axis_name),
         label=b_label,
         name=b_name,
         storage=hist.storage.Weight()
         )

def TH2F(name, title, x_nbins, x_bin_low, x_bin_high, y_nbins, y_bin_low, y_bin_high):
    b_x_axis_name = 'X'
    b_y_axis_name = 'Y'
    title_split = title.split(';')
    if len(title_split) > 1:
        b_x_axis_name = title_split[1]
    if len(title_split) > 2:
        b_y_axis_name = title_split[2]
    b_name = title_split[0]
    b_label = name
    
    return hist.dask.Hist(
        hist.axis.Regular(bins=x_nbins, start=x_bin_low, stop=x_bin_high, name=b_x_axis_name), 
        hist.axis.Regular(bins=y_nbins, start=y_bin_low, stop=y_bin_high, name=b_y_axis_name), 
        label=b_label, 
        name=b_name,
        storage=hist.storage.Weight()
        )

def fill_1Dhist(hist, array, weights=None):
    flar = ak.drop_none(ak.flatten(array))
    
    if weights is None:
        hist.fill(flar, threads=None)
        # ROOT.fill_1Dhist(hist=hist, array=flar)
    else:
        hist.fill(flar, weights)
        # ROOT.fill_1Dhist(hist=hist, array=flar, weights=weights)
        
def fill_2Dhist(hist, arrayX, arrayY, weights=None):
    flar_x = ak.drop_none(ak.flatten(arrayX))
    flar_y = ak.drop_none(ak.flatten(arrayY))

    if weights is None:
        # ROOT.fill_2Dhist(hist=hist, arrayX=flar_x, arrayY=flar_y)
        hist.fill(flar_x, flar_y, threads=None)
    else:
        # ROOT.fill_2Dhist(hist=hist, arrayX=flar_x, arrayY=flar_y, weights=weights)
        hist.fill(flar_x, flar_y, weights)

# 9. plotters.py

In [ ]:
"""
Definines and instantiate the plotter classes.

These are the classes where the analysis logic is implemented.
The plotter classes need to implement a standard interface:
init:
    accept one or more DFCollection (or TPSet) and a corresponding list of selections.

book_histos:
    activates the relevant DFCollection instances and books the HistoClasses defined in the
    `l1THistos` module for each combination of the object and selections.
     The naming convention of the histograms uses the namse of the objects,
     the name of the selection and the name of the gen-matched object (if any).

fill_histos:
    actually implements the analysis logic, running the selections on the input
    collections and filling the histograms.

Several collections of plotters are also instantiated. Which one will actually be run
is steered via the configuration file.
"""
import awkward as ak
import numpy as np
import pandas as pd
import ROOT

import python.calibrations as calib
import python.clusterTools as clAlgo
import python.selections as selections
import python.utils as utils


# import collections as collections
# ROOT.gROOT.ProcessLine('#include "src/fastfilling.h"')


class Test:
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return f'<TEST class, {self.name}>'


class BasePlotter:
    def __init__(self, data_set, data_selections, gen_set=None, gen_selections=None):
        self.data_set = data_set
        self.data_selections = data_selections
        self.gen_set = gen_set
        self.gen_selections = gen_selections

    @property
    def tp_set(self):
        return self.data_set

    @property
    def tp_selections(self):
        return self.data_selections

    def get_histo_primitives(self):
        histo_primitives = pd.DataFrame(columns=['data', 'data_sel', 'gen_sel', 'data_label', 'data_sel_label', 'gen_sel_label'])
        gen_sel_names = ['nomatch']
        gen_sel_labels = ['']
        if self.gen_selections is not None:
            gen_sel_names = [sel.name for sel in self.gen_selections]
            gen_sel_labels = [sel.label for sel in self.gen_selections]

        for data_sel in self.data_selections:
            for idx, gen_sel_name in enumerate(gen_sel_names):
                histo_primitives = histo_primitives.append({'data': self.data_set.name,
                                                            'data_sel': data_sel.name,
                                                            'gen_sel': gen_sel_name,
                                                            'data_label': self.data_set.label,
                                                            'data_sel_label': data_sel.label,
                                                            'gen_sel_label': gen_sel_labels[idx]},
                                                           ignore_index=True)
        return histo_primitives

    def fill_histos(self, debug=0):
        pass

    def fill_histos_event(self, idx, debug=0):
        printt("[JUYPTER] FILL HISTOS EVENT WORKING")
        if self.data_set.new_read:
            self.fill_histos(debug)

    def __repr__(self):
        return f'<{self.__class__.__name__}, ds: {self.data_set}, ds_sel: {self.data_selections}, g: {self.gen_set}, g_sel: {self.gen_selections} >'
    # def change_genpart_selection(self, newselection):
    #     """Allow customization of gen selection per sample."""
    #     if self.gen_selections is not None:
    #         self.gen_selections = newselection
    #
    # def __repr__(self):
    #     if  self.gen_selections is not None:
    #         return '<{}, tp: {}, gen_sel: {}>'.format(self.__class__.__name__,
    #                                                   self.data_set.name,
    #                                                   self.gen_selections[0].selection)
    #     else:
    #         return '<{}, tp: {}>'.format(self.__class__.__name__,
    #                                      self.data_set.name)


class HGCCl3DRatePlotter(BasePlotter):
    def __init__(self, tp_set, tp_selections=[selections.Selection('all')]):
        self.h_rate = {}
        super(HGCCl3DRatePlotter, self).__init__(tp_set, tp_selections)

    def book_histos(self):
        self.tp_set.activate()
        tp_name = self.tp_set.name
        for selection in self.tp_selections:
            self.h_rate[selection.name] = RateHistos(name=f'{tp_name}_{selection.name}')

    def fill_histos(self, debug=0):
        # print '------------------'
        # print self.tp_set.name
        for selection in self.tp_selections:
            sel_clusters = self.tp_set.query(selection)
            self.h_rate[selection.name].fill_many(sel_clusters.loc[sel_clusters['pt_em'].groupby(level='entry', group_keys=False).nlargest(n=1).index, ['pt_em']])
            self.h_rate[selection.name].fill_norm(self.tp_set.new_read_nentries)


class GenericDataFramePlotter(BasePlotter):
    def __init__(self, HistoClass, data_set, selections=[selections.Selection('all')], pt_bins=None):
        self.HistoClass = HistoClass
        self.h_set = {}
        self.pt_bins = pt_bins
        super(GenericDataFramePlotter, self).__init__(data_set, selections)

    def book_histos(self):
        self.data_set.activate()
        data_name = self.data_set.name
        for selection in self.data_selections:
            if self.pt_bins:
                self.h_set[selection.name] = self.HistoClass(
                    name=f'{data_name}_{selection.name}_nomatch',
                    pt_bins=self.pt_bins)
            else:
                self.h_set[selection.name] = self.HistoClass(
                    name=f'{data_name}_{selection.name}_nomatch')


    def fill_histos(self, debug=0):
        for data_sel in self.data_selections:
            data = self.data_set.df
            if not data_sel.all:
                data = data[data_sel.selection(data)]
            self.h_set[data_sel.name].fill(data)


class GenPlotter(GenericDataFramePlotter):
    def __init__(self, gen_set, gen_selections=[selections.Selection('all')], pt_bins=None):
        super(GenPlotter, self).__init__(
            GenParticleExtraHistos,
            gen_set,
            gen_selections,
            pt_bins)


class TkElePlotter(GenericDataFramePlotter):
    def __init__(self, tkeg_set, tkeg_selections=[selections.Selection('all')]):
        super(TkElePlotter, self).__init__(TkEleHistos, tkeg_set, tkeg_selections)


class TkEmPlotter(GenericDataFramePlotter):
    def __init__(self, tkeg_set, tkeg_selections=[selections.Selection('all')]):
        super(TkEmPlotter, self).__init__(TkEmHistos, tkeg_set, tkeg_selections)


class TkEGPlotter(GenericDataFramePlotter):
    def __init__(self, tkeg_set, tkeg_selections=[selections.Selection('all')]):
        super(TkEGPlotter, self).__init__(TkEGHistos, tkeg_set, tkeg_selections)


class TrackPlotter(GenericDataFramePlotter):
    def __init__(self, trk_set, track_selections=[selections.Selection('all')]):
        super(TrackPlotter, self).__init__(TrackHistos, trk_set, track_selections)


class EGPlotter(GenericDataFramePlotter):
    def __init__(self, eg_set, eg_selections=[selections.Selection('all')]):
        super(EGPlotter, self).__init__(EGHistos, eg_set, eg_selections)


class TTPlotter(GenericDataFramePlotter):
    def __init__(self, tt_set, tt_selections=[selections.Selection('all')]):
        super(TTPlotter, self).__init__(TriggerTowerHistos, tt_set, tt_selections)


class DecTkPlotter(GenericDataFramePlotter):
    def __init__(self, tk_set, tk_selections=[selections.Selection('all')]):
        super(DecTkPlotter, self).__init__(DecTkHistos, tk_set, tk_selections)

class Cl3DPlotter(GenericDataFramePlotter):
    def __init__(self, data_set, data_selections=[selections.Selection('all')]):
        super(Cl3DPlotter, self).__init__(Cluster3DHistos, data_set, data_selections)

class TPPlotter(BasePlotter):
    def __init__(self, tp_set, tp_selections=[selections.Selection('all')]):
        # self.tp_set = tp_set
        # self.tp_selections = tp_selections
        self.h_tpset = {}
        super(TPPlotter, self).__init__(tp_set, tp_selections)

    def book_histos(self):
        self.tp_set.activate()
        tp_name = self.tp_set.name
        for selection in self.tp_selections:
            self.h_tpset[selection.name] = HistoSetClusters(name=f'{tp_name}_{selection.name}_nomatch')

    def fill_histos(self, debug=0):
        # FIXME: migrate to the new query caching system
        for tp_sel in self.tp_selections:
            tcs = self.tp_set.tc_df
            cl2Ds = self.tp_set.cl2d_df
            cl3Ds = self.tp_set.cl3d_df
            if not tp_sel.all:
                cl3Ds = self.tp_set.cl3d_df.query(tp_sel.selection)
            # debug = 4
            # utils.debugPrintOut(debug, '{}_{}'.format(self.tp_set.name, 'TCs'), tcs, tcs[:3])
            # utils.debugPrintOut(debug, '{}_{}'.format(self.tp_set.name, 'CL2D'), cl2Ds, cl2Ds[:3])
            # utils.debugPrintOut(debug, '{}_{}'.format(self.tp_set.name, 'CL3D'), cl3Ds, cl3Ds[:3])
            if not cl3Ds.empty and not cl2Ds.empty and not tcs.empty:
                self.h_tpset[tp_sel.name].fill(tcs, cl2Ds, cl3Ds)


class TPGenMatchPlotter(BasePlotter):
    def __init__(self, tp_set, gen_set,
                 tp_selections=[selections.Selection('all')],
                 gen_selections=[selections.Selection('all')],
                 extended_range=False):
        # self.tp_set = tp_set
        # self.tp_selections = tp_selections
        # self.gen_set = gen_set
        # self.gen_selections = gen_selections
        self.h_tpset = {}
        self.h_resoset = {}
        self.h_effset = {}
        self.h_conecluster = {}
        self.extended_range = extended_range
        super(TPGenMatchPlotter, self).__init__(
            tp_set,
            tp_selections,
            gen_set,
            selections.multiply_selections(
                gen_selections,
                [selections.Selection('', '', 'gen > 0')]))

    def plot3DClusterMatch(self,
                           genParticles,
                           trigger3DClusters,
                           triggerClusters,
                           triggerCells,
                           histoGen,
                           histoGenMatched,
                           histoTCMatch,
                           histoClMatch,
                           histo3DClMatch,
                           histoReso,
                           histoResoCone,
                           histoReso2D,
                           histoConeClusters,
                           algoname,
                           debug):
        def computeIsolation(all3DClusters, idx_best_match, idx_incone, dr):
            ret = pd.DataFrame()
            # print 'index best match: {}'.format(idx_best_match)
            # print 'indexes all in cone: {}'.format(idx_incone)
            components = all3DClusters[(all3DClusters.index.isin(idx_incone)) & ~(all3DClusters.index == idx_best_match)]
            # print 'components indexes: {}'.format(components.index)
            compindr = components[np.sqrt((components.eta-all3DClusters.loc[idx_best_match].eta)**2 + (components.phi-all3DClusters.loc[idx_best_match].phi)**2) < dr]
            if not compindr.empty:
                # print 'components indexes in dr: {}'.format(compindr.index)
                ret['energy'] = [compindr.energy.sum()]
                ret['eta'] = [np.sum(compindr.eta*compindr.energy)/compindr.energy.sum()]
                ret['pt'] = [(ret.energy/np.cosh(ret.eta)).values[0]]
            else:
                ret['energy'] = [0.]
                ret['eta'] = [0.]
                ret['pt'] = [0.]
            return ret

        def sumClustersInCone(all3DClusters, idx_incone, debug=0):
            components = all3DClusters[all3DClusters.index.isin(idx_incone)]
            ret = clAlgo.sum3DClusters(components)
            if debug > 0:
                print('-------- in cone:')
                print(components.sort_values(by='pt', ascending=False))
                print('   - Cone sum:')
                print(ret)
            return ret

        best_match_indexes = {}
        if not trigger3DClusters.empty:
            best_match_indexes, allmatches = utils.match_etaphi(genParticles[['eta', 'phi']],
                                                                trigger3DClusters[['eta', 'phi']],
                                                                trigger3DClusters['pt'],
                                                                deltaR=0.1)
        # print ('------ best match: ')
        # print (best_match_indexes)
        # print ('------ all matches:')
        # print (allmatches)

        # allmatched2Dclusters = list()
        # matchedClustersAll = pd.DataFrame()
        if histoGen is not None:
            histoGen.fill(genParticles)

        for idx, genParticle in genParticles.iterrows():
            if idx in best_match_indexes.keys():
                # print ('-----------------------')
                #  print(genParticle)
                matched3DCluster = trigger3DClusters.loc[[best_match_indexes[idx]]]
                # print (matched3DCluster)
                # allMatches = trigger3DClusters.iloc[allmatches[idx]]
                # print ('--')
                # print (allMatches)
                # print (matched3DCluster.clusters.item())
                # print (type(matched3DCluster.clusters.item()))
                # matchedClusters = triggerClusters[ [x in matched3DCluster.clusters.item() for x in triggerClusters.id]]
                matchedClusters = triggerClusters[triggerClusters.id.isin(matched3DCluster.clusters.item())]
                # print (matchedClusters)
                matchedTriggerCells = triggerCells[triggerCells.id.isin(np.concatenate(matchedClusters.cells.values))]
                # allmatched2Dclusters. append(matchedClusters)

                if False:
                    if 'energyCentral' not in matched3DCluster.columns:
                        calib_factor = 1.084
                        matched3DCluster['energyCentral'] = [matchedClusters[(matchedClusters.layer > 9) & (matchedClusters.layer < 21)].energy.sum()*calib_factor]

                if False:
                    iso_df = computeIsolation(trigger3DClusters,
                                              idx_best_match=best_match_indexes[idx],
                                              idx_incone=allmatches[idx], dr=0.2)
                    matched3DCluster['iso0p2'] = iso_df.energy
                    matched3DCluster['isoRel0p2'] = iso_df.pt/matched3DCluster.pt

                # fill the plots
                histoTCMatch.fill(matchedTriggerCells)
                histoClMatch.fill(matchedClusters)
                histo3DClMatch.fill(matched3DCluster)

                # print matchedClusters
                # print matchedClusters.layer.unique()
                for layer in matchedClusters.layer.unique():
                    if histoReso2D is not None:
                        histoReso2D.fill(reference=genParticle,
                                         target=clAlgo.build2D(matchedClusters[matchedClusters.layer == layer]))

                if False:
                    histoReso2D.fill(reference=genParticle, target=matchedClusters)
                histoReso.fill(reference=genParticle, target=matched3DCluster)

                if False:
                    # now we fill the reso plot for all the clusters in the cone
                    clustersInCone = sumClustersInCone(trigger3DClusters, allmatches[idx])

                    def fill_cluster_incone_histos(cl3ds,
                                                   idx_allmatches,
                                                   idx_bestmatch,
                                                   charge,
                                                   h_clustersInCone,
                                                   debug=0):
                        if debug > 4:
                            print(f'- best match: {idx_bestmatch}, all matches: {idx_allmatches}')
                        bestcl = cl3ds.loc[idx_bestmatch]
                        h_clustersInCone.fill_n(len(idx_allmatches)-1)
                        for idx in idx_allmatches:
                            if idx == idx_bestmatch:
                                continue
                            clincone = cl3ds.loc[idx]
                            h_clustersInCone.fill(reference=bestcl,
                                                  target=clincone,
                                                  charge=charge)
                    # print genParticle
                    # print genParticle.pid/abs(genParticle.pid)
                    fill_cluster_incone_histos(trigger3DClusters,
                                               allmatches[idx],
                                               best_match_indexes[idx],
                                               genParticle.pid/abs(genParticle.pid),
                                               histoConeClusters)

                    # print ('----- in cone sum:')
                    # print (clustersInCone)
                    # histoResoCone.fill(reference=genParticle, target=clustersInCone.iloc[0])

                if histoGenMatched is not None:
                    histoGenMatched.fill(genParticles.loc[[idx]])

                if debug >= 6:
                    print(f'--- Dump match for algo {algoname} ---------------')
                    print(f'GEN particle: idx: {idx}')
                    print(genParticle)
                    print('Matched to 3D cluster:')
                    print(matched3DCluster)
                    print('Matched 2D clusters:')
                    print(matchedClusters)
                    print('matched cells:')
                    print(matchedTriggerCells)

                    print(f'3D cluster energy: {matched3DCluster.energy.sum()}')
                    print(f'3D cluster pt: {matched3DCluster.pt.sum()}')
                    calib_factor = 1.084
                    print(f'sum 2D cluster energy: {matchedClusters.energy.sum() * calib_factor}')
                    # print ('sum 2D cluster pt: {}'.format(matchedClusters.pt.sum()*calib_factor))
                    print(f'sum TC energy: {matchedTriggerCells.energy.sum()}')
                    print('Sum of matched clusters in cone:')
                    print(clustersInCone)
            elif debug >= 5:
                print(f'==== Warning no match found for algo {algoname}, idx {idx} ======================')
                if debug >= 2:
                    print(genParticle)
                    print(trigger3DClusters)

        # if len(allmatched2Dclusters) != 0:
        #     matchedClustersAll = pd.concat(allmatched2Dclusters)
        # return matchedClustersAll

    def book_histos(self):
        self.gen_set.activate()
        self.tp_set.activate()
        for tp_sel in self.tp_selections:
            for gen_sel in self.gen_selections:
                histo_name = f'{self.tp_set.name}_{tp_sel.name}_{gen_sel.name}'
                self.h_tpset[histo_name] = HistoSetClusters(histo_name)
                self.h_resoset[histo_name] = HistoSetReso(histo_name)
                self.h_effset[histo_name] = HistoSetEff(histo_name, extended_range=self.extended_range)
                self.h_conecluster[histo_name] = ClusterConeHistos(histo_name)

    def fill_histos(self, debug=0):
        for tp_sel in self.data_selections:
            tcs = self.tp_set.tc_df
            cl2Ds = self.tp_set.cl2d_df
            cl3Ds = self.tp_set.cl3ds.query(tp_sel)
            for gen_sel in self.gen_selections:
                genReference = self.gen_set.query(gen_sel)
                histo_name = f'{self.tp_set.name}_{tp_sel.name}_{gen_sel.name}'

                h_tpset_match = self.h_tpset[histo_name]
                h_resoset = self.h_resoset[histo_name]
                h_genseleff = self.h_effset[histo_name]
                h_conecl = self.h_conecluster[histo_name]
                # print 'TPsel: {}, GENsel: {}'.format(tp_sel.name, gen_sel.name)
                # print cl3Ds
                # print cl2Ds
                # print tcs
                self.plot3DClusterMatch(genReference,
                                        cl3Ds,
                                        cl2Ds,
                                        tcs,
                                        h_genseleff.h_den,
                                        h_genseleff.h_num,
                                        h_tpset_match.htc,
                                        h_tpset_match.hcl2d,
                                        h_tpset_match.hcl3d,
                                        h_resoset.hreso,
                                        h_resoset.hresoCone,
                                        h_resoset.hreso2D,
                                        h_conecl,
                                        self.tp_set.name,
                                        debug)

    def __repr__(self):
        return f'<{self.__class__.__name__} tps: {self.tp_set.name}, tps_s: {[sel.name for sel in self.tp_selections]}, gen:{self.gen_set.name}, gen_s:{[sel.name for sel in self.gen_selections]}> '


class GenericGenMatchPlotter(BasePlotter):
    def __init__(self, ObjectHistoClass, ResoHistoClass,
                 data_set, gen_set,
                 data_selections=[selections.Selection('all')],
                 gen_selections=[selections.Selection('all')],
                 gen_eta_phi_columns=('caloeta', 'calophi'),
                 drcut=0.1,
                 pt_bins=None):
        self.ObjectHistoClass = ObjectHistoClass
        self.ResoHistoClass = ResoHistoClass
        # self.data_set = data_set
        # self.data_selections = data_selections
        # self.gen_set = gen_set
        # self.gen_selections = gen_selections
        self.h_dataset = {}
        self.h_resoset = {}
        self.h_effset = {}
        self.gen_eta_phi_columns = gen_eta_phi_columns
        self.dr2 = drcut*drcut
        self.pt_bins = pt_bins

        super(GenericGenMatchPlotter, self).__init__(
            data_set,
            data_selections,
            gen_set,
            gen_selections)

        # print self
        # print gen_selections

    def plotObjectMatch(self,
                        gen,
                        objects,
                        h_gen,
                        h_gen_matched,
                        h_object_matched,
                        h_reso,
                        algoname,
                        debug):


        # fill histo with all selected GEN particles before any match
        h_gen.fill(gen)

        # perform the matching
        match_eta = ak.cartesian([objects.eta, gen[self.gen_eta_phi_columns[0]]])
        match_phi = ak.cartesian([objects.phi, gen[self.gen_eta_phi_columns[1]]])
        match_pt = ak.cartesian([objects.pt, gen.pt])
        match_idx = ak.argcartesian([objects.eta, gen.eta])

        obj_eta, gen_eta = ak.unzip(match_eta)
        obj_phi, gen_phi = ak.unzip(match_phi)
        obj_pt, gen_pt = ak.unzip(match_pt)
        obj_idx, gen_idx = ak.unzip(match_idx)
        dpt = np.abs(obj_pt - gen_pt)
        dr2 = (obj_eta-gen_eta)**2+(obj_phi-gen_phi)**2
        match = ak.Array(data={'ele_idx': obj_idx, 'gen_idx': gen_idx, 'dpt': dpt, 'dr2': dr2})
        dr_match=match[match.dr2<self.dr2]
        for genid in np.unique(ak.flatten(dr_match.gen_idx)):
            gen_match_id = dr_match[dr_match.gen_idx == genid]
            dpt_min_index = ak.argmin(gen_match_id.dpt, axis=1, keepdims=True)
            best_match_id = gen_match_id[dpt_min_index]
            matched_obj = objects[best_match_id.ele_idx]
            matched_gen = gen[best_match_id.gen_idx]
            h_object_matched.fill(matched_obj)
            if h_gen_matched is not None:
                h_gen_matched.fill(matched_gen)
            h_reso.fill(reference=matched_gen,
                        target=matched_obj)
            # FIXME: [AWKWARD]
            # if hasattr(h_reso, 'fill_nMatch'):
            #     h_reso.fill_nMatch(len(allmatches[idx]))


    def book_histos(self):
        self.gen_set.activate()
        self.data_set.activate()
        # print(f'# data sel: {len(self.data_selections)} x # gen sel: {len(self.gen_selections)} = {len(self.data_selections)*len(self.gen_selections)}')
        for tp_sel in self.data_selections:
            for gen_sel in self.gen_selections:
                histo_name = f'{self.data_set.name}_{tp_sel.name}_{gen_sel.name}'
                self.h_dataset[histo_name] = self.ObjectHistoClass(histo_name)
                self.h_resoset[histo_name] = self.ResoHistoClass(histo_name)
                self.h_effset[histo_name] = HistoSetEff(histo_name, pt_bins=self.pt_bins)

    def fill_histos(self, debug=0):
        # FIXME: we need to reduce the # of jugged dimensions for the selection slicing to work in AWKWARD....
        # print(self.gen_set.df.fields)
        # gen = self.gen_set.df[['eta', 'abseta', 'phi', 'pt', 'energy', 'exeta', 'exphi', 'fbrem', 'gen', 'pid', 'reachedEE', 'pdgid', 'ovx', 'ovy', 'ovz']]
        gen = self.gen_set.df

        for tp_sel in self.data_selections:
            # print(tp_sel)
            if tp_sel.all:
                # FIXME: workaround for now
                objects = self.data_set.df
            else:
                objects = self.data_set.df[tp_sel.selection(self.data_set.df)]
            for gen_sel in self.gen_selections:
                # print(gen_sel)
                # print(gen_sel.selection(gen))
                if gen_sel.all:
                    genReference = gen
                else:
                    genReference = gen[gen_sel.selection(gen)]
                histo_name = f'{self.data_set.name}_{tp_sel.name}_{gen_sel.name}'
                # print (histo_name)
                # print (f'# data: {objects.shape[0]}')
                # print (f'# gen: {genReference.shape[0]}')
                # print (genReference)
                h_obj_match = self.h_dataset[histo_name]
                h_resoset = self.h_resoset[histo_name]
                h_genseleff = self.h_effset[histo_name]
                self.plotObjectMatch(genReference,
                                     objects,
                                     h_genseleff.h_den,
                                     h_genseleff.h_num,
                                     h_obj_match,
                                     h_resoset,
                                     self.data_set.name,
                                     debug)


class TrackGenMatchPlotter(GenericGenMatchPlotter):
    def __init__(self, data_set, gen_set,
                 data_selections=[selections.Selection('all')],
                 gen_selections=[selections.Selection('all')]):
        super(TrackGenMatchPlotter, self).__init__(TrackHistos, TrackResoHistos,
                                                   data_set, gen_set,
                                                   data_selections, gen_selections,
                                                   gen_eta_phi_columns=['eta', 'phi'])


class JetGenMatchPlotter(GenericGenMatchPlotter):
    def __init__(self, data_set, gen_set,
                 data_selections=[selections.Selection('all')],
                 gen_selections=[selections.Selection('all')]):
        super(JetGenMatchPlotter, self).__init__(JetHistos, JetResoHistos,
                                                   data_set, gen_set,
                                                   data_selections, gen_selections,
                                                   gen_eta_phi_columns=['eta', 'phi'],
                                                   drcut=0.3)


class TrackGenMatchPlotter(GenericGenMatchPlotter):
    def __init__(self, data_set, gen_set,
                 data_selections=[selections.Selection('all')],
                 gen_selections=[selections.Selection('all')]):
        super(TrackGenMatchPlotter, self).__init__(TrackHistos, TrackResoHistos,
                                                   data_set, gen_set,
                                                   data_selections, gen_selections,
                                                   gen_eta_phi_columns=['eta', 'phi'])


class DecTrackGenMatchPlotter(GenericGenMatchPlotter):
    def __init__(self, data_set, gen_set,
                 data_selections=[selections.Selection('all')],
                 gen_selections=[selections.Selection('all')]):
        super(DecTrackGenMatchPlotter, self).__init__(
            DecTkHistos, DecTkResoHistos,
            data_set, gen_set,
            data_selections, gen_selections,
            gen_eta_phi_columns=['eta', 'phi'])


class Cl3DGenMatchPlotter(GenericGenMatchPlotter):
    def __init__(self, data_set, gen_set,
                 data_selections=[selections.Selection('all')],
                 gen_selections=[selections.Selection('all')],
                 pt_bins=None):
        super(Cl3DGenMatchPlotter, self).__init__(Cluster3DHistos, ResoHistos,
                                                  data_set, gen_set,
                                                  data_selections, gen_selections,
                                                  gen_eta_phi_columns=['caloeta', 'calophi'],
                                                  pt_bins=pt_bins)


class EGGenMatchPtWPSPlotter(GenericGenMatchPlotter):
    def __init__(self, data_set, gen_set, gen_selections):
        super(EGGenMatchPtWPSPlotter, self).__init__(
            EGHistos, EGResoHistos,
            data_set, gen_set,
            [], gen_selections)

    def book_histos(self):
        calib_mgr = calib.CalibManager()
        rate_pt_wps = calib_mgr.get_pt_wps()
        self.data_selections = calib.rate_pt_wps_selections(
            rate_pt_wps, self.data_set.name)
        GenericGenMatchPlotter.book_histos(self)


class HGCCl3DGenMatchPtWPSPlotter(GenericGenMatchPlotter):
    def __init__(self, data_set, gen_set, gen_selections):
        super(HGCCl3DGenMatchPtWPSPlotter, self).__init__(
            Cluster3DHistos, ResoHistos,
            data_set, gen_set,
            [], gen_selections)

    def book_histos(self):
        calib_mgr = calib.CalibManager()
        rate_pt_wps = calib_mgr.get_pt_wps()
        self.data_selections = calib.rate_pt_wps_selections(
            rate_pt_wps, self.data_set.name, 'pt_em')
        GenericGenMatchPlotter.book_histos(self)


class EGGenMatchPlotter(GenericGenMatchPlotter):
    def __init__(self, data_set, gen_set,
                 data_selections=[selections.Selection('all')],
                 gen_selections=[selections.Selection('all')],
                 gen_eta_phi_columns=('caloeta', 'calophi'),
                 pt_bins=None):
        super(EGGenMatchPlotter, self).__init__(EGHistos, EGResoHistos,
                                                data_set, gen_set,
                                                data_selections, gen_selections,
                                                gen_eta_phi_columns=gen_eta_phi_columns,
                                                pt_bins=pt_bins)


class TkEGGenMatchPlotter(GenericGenMatchPlotter):
    def __init__(self, data_set, gen_set,
                 data_selections=[selections.Selection('all')],
                 gen_selections=[selections.Selection('all')]):
        super(TkEGGenMatchPlotter, self).__init__(TkEGHistos, EGResoHistos,
                                                  data_set, gen_set,
                                                  data_selections, gen_selections)


class ResoNtupleMatchPlotter(GenericGenMatchPlotter):
    def __init__(self, data_set, gen_set,
                 data_selections=[selections.Selection('all')],
                 gen_selections=[selections.Selection('all')]):
        super(ResoNtupleMatchPlotter, self).__init__(EGHistos, ResoTuples,
                                                data_set, gen_set,
                                                data_selections, gen_selections, drcut=0.2)



# class ResoNtupleMatchPlotter(BasePlotter):
#     def __init__(self, data_set, gen_set,
#                  data_selections=[selections.Selection('all')], gen_selections=[selections.Selection('all')]):

#         self.h_calibration = {}
#         super(ResoNtupleMatchPlotter, self).__init__(
#             data_set,
#             data_selections,
#             gen_set,
#             selections.multiply_selections(
#                 gen_selections,
#                 [selections.Selection('', '', 'gen > 0')]))

#         # print self
#         # print gen_selections

#     def plotObjectMatch(self,
#                         genParticles,
#                         objects,
#                         h_calibration,
#                         algoname,
#                         debug):
#         best_match_indexes = {}
#         if not objects.empty:
#             best_match_indexes, allmatches = utils.match_etaphi(genParticles[['exeta', 'exphi']],
#                                                                 objects[['eta', 'phi']],
#                                                                 objects['pt'],
#                                                                 deltaR=0.1)

#         for idx, genParticle in genParticles.iterrows():
#             if idx in best_match_indexes.keys():
#                 # print ('-----------------------')
#                 #  print(genParticle)
#                 obj_matched = objects.loc[[best_match_indexes[idx]]]
#                 # print obj_matched
#                 # print obj_matched.clusters
#                 # print obj_matched.clusters[0]
#                 # print algoname
#                 # print obj_matched[['energy', 'layer_energy']]
#                 h_calibration.fill(reference=genParticle, target=obj_matched)

#                 if debug >= 4:
#                     print(('--- Dump match for algo {} ---------------'.format(algoname)))
#                     print(('GEN particle: idx: {}'.format(idx)))
#                     print(genParticle)
#                     print('Matched to track object:')
#                     print(obj_matched)
#             else:
#                 if debug >= 5:
#                     print(('==== Warning no match found for algo {}, idx {} ======================'.format(algoname, idx)))
#                     print(genParticle)
#                     print(objects)

#     def book_histos(self):
#         self.gen_set.activate()
#         self.data_set.activate()
#         for tp_sel in self.data_selections:
#             for gen_sel in self.gen_selections:
#                 histo_name = '{}_{}_{}'.format(self.data_set.name, tp_sel.name, gen_sel.name)
#                 self.h_calibration[histo_name] = ResoTuples(histo_name)

#     def fill_histos(self, debug=0):
#         for tp_sel in self.data_selections:
#             objects = self.data_set.query(tp_sel)
#             for gen_sel in self.gen_selections:
#                 genReference = self.gen_set.query(gen_sel)
#                 histo_name = '{}_{}_{}'.format(self.data_set.name, tp_sel.name, gen_sel.name)

#                 h_calib = self.h_calibration[histo_name]
#                 # print 'TPsel: {}, GENsel: {}'.format(tp_sel.name, gen_sel.name)
#                 self.plotObjectMatch(genReference,
#                                      objects,
#                                      h_calib,
#                                      self.data_set.name,
#                                      debug)


class CalibrationPlotter(BasePlotter):
    def __init__(self, data_set, gen_set,
                 data_selections=[selections.Selection('all')], gen_selections=[selections.Selection('all')]):

        self.h_calibration = {}
        super(CalibrationPlotter, self).__init__(
            data_set,
            data_selections,
            gen_set,
            selections.multiply_selections(
                gen_selections,
                [selections.Selection('', '', 'gen > 0')]))

        # print self
        # print gen_selections

    def plotObjectMatch(self,
                        genParticles,
                        objects,
                        tcs,
                        h_calibration,
                        algoname,
                        debug):
        best_match_indexes = {}
        if not objects.empty:
            best_match_indexes, allmatches = utils.match_etaphi(genParticles[['exeta', 'exphi']],
                                                                objects[['eta', 'phi']],
                                                                objects['pt'],
                                                                deltaR=0.1)

        for idx, genParticle in genParticles.iterrows():
            if idx in best_match_indexes.keys():
                # print ('-----------------------')
                #  print(genParticle)
                obj_matched = objects.loc[[best_match_indexes[idx]]]
                # print obj_matched
                # print obj_matched.clusters
                # print obj_matched.clusters[0]
                components = tcs[tcs.id.isin(obj_matched.iloc[0].clusters)].copy()
                if 'layer_energy' not in obj_matched.columns:
                    layer_energy = []
                    for layer in range(1, 29, 2):
                        # components[components.layer == layer].energy.sum()
                        layer_energy.append(components[components.layer == layer].energy.sum())
                    obj_matched['layer_energy'] = [layer_energy]

                # print algoname
                # print obj_matched[['energy', 'layer_energy']]
                h_calibration.fill(reference=genParticle, target=obj_matched)

                if debug >= 4:
                    print(f'--- Dump match for algo {algoname} ---------------')
                    print(f'GEN particle: idx: {idx}')
                    print(genParticle)
                    print('Matched to track object:')
                    print(obj_matched)
            elif debug >= 5:
                print(f'==== Warning no match found for algo {algoname}, idx {idx} ======================')
                print(genParticle)
                print(objects)

    def book_histos(self):
        self.gen_set.activate()
        self.data_set.activate()
        for tp_sel in self.data_selections:
            for gen_sel in self.gen_selections:
                histo_name = f'{self.data_set.name}_{tp_sel.name}_{gen_sel.name}'
                self.h_calibration[histo_name] = CalibrationHistos(histo_name)

    def fill_histos(self, debug=0):
        for tp_sel in self.data_selections:
            objects = self.data_set.query(tp_sel)
            for gen_sel in self.gen_selections:
                genReference = self.gen_set.query(gen_sel)
                histo_name = f'{self.data_set.name}_{tp_sel.name}_{gen_sel.name}'

                h_calib = self.h_calibration[histo_name]
                # print 'TPsel: {}, GENsel: {}'.format(tp_sel.name, gen_sel.name)
                self.plotObjectMatch(genReference,
                                     objects,
                                     self.data_set.tcs.df,
                                     h_calib,
                                     self.data_set.name,
                                     debug)


class TTGenMatchPlotter:
    def __init__(self, tt_set, gen_set,
                 tt_selections=[selections.Selection('all')], gen_selections=[selections.Selection('all')]):
        self.tt_set = tt_set
        self.tt_selections = tt_selections
        self.gen_set = gen_set
        self.gen_selections = gen_selections
        self.h_tt = {}
        self.h_reso_tt = {}
        self.h_reso_ttcl = {}

    def book_histos(self):
        self.tt_set.activate()
        self.gen_set.activate()
        for tp_sel in self.tt_selections:
            for gen_sel in self.gen_selections:
                histo_name = f'{tp_sel.name}_{gen_sel.name}'
                self.h_tt[histo_name] = TriggerTowerHistos(f'{self.tt_set.name}_{histo_name}')
                self.h_reso_tt[histo_name] = TriggerTowerResoHistos(f'{self.tt_set.name}_{histo_name}')
                self.h_reso_ttcl[histo_name] = TriggerTowerResoHistos(f'{self.tt_set.name}Cl_{histo_name}')

    def fill_histos(self, debug=0):
        triggerTowers_all = self.tt_set.df
        genParts_all = self.gen_set.df[(self.gen_set.df.gen > 0)]
        for tp_sel in self.tt_selections:
            triggerTowers = triggerTowers_all
            if not tp_sel.all:
                triggerTowers = triggerTowers_all.query(tp_sel.selection)
            for gen_sel in self.gen_selections:
                histo_name = f'{tp_sel.name}_{gen_sel.name}'
                genReference = genParts_all
                if not gen_sel.all:
                    genReference = genParts_all.query(gen_sel.selection)
                self.plotTriggerTowerMatch(genReference,
                                           None,
                                           triggerTowers,
                                           self.h_tt[histo_name],
                                           self.h_reso_tt[histo_name],
                                           self.h_reso_ttcl[histo_name],
                                           'TThighestPt',
                                           debug)

    def plotTriggerTowerMatch(self,
                              genParticles,
                              histoGen,
                              triggerTowers,
                              histoTowersMatch,
                              histoTowersReso,
                              histoTowersResoCl,
                              algoname,
                              debug):

        best_match_indexes = {}
        if triggerTowers.shape[0] != 0:
            best_match_indexes, allmatches = utils.match_etaphi(genParticles[['exeta', 'exphi']],
                                                                triggerTowers[['eta', 'phi']],
                                                                triggerTowers['pt'],
                                                                deltaR=0.2)
            # print ('-----------------------')
            # print (best_match_indexes)
        # print ('------ best match: ')
        # print (best_match_indexes)
        # print ('------ all matches:')
        # print (allmatches)

        if histoGen is not None:
            histoGen.fill(genParticles)

        for idx, genParticle in genParticles.iterrows():
            if idx in best_match_indexes.keys():
                # print ('-----------------------')
                #  print(genParticle)
                matchedTower = triggerTowers.loc[[best_match_indexes[idx]]]
                # print (matched3DCluster)
                # allMatches = trigger3DClusters.iloc[allmatches[idx]]
                # print ('--')
                # print (allMatches)
                # print (matched3DCluster.clusters.item())
                # print (type(matched3DCluster.clusters.item()))
                # matchedClusters = triggerClusters[ [x in matched3DCluster.clusters.item() for x in triggerClusters.id]]

                # fill the plots
                histoTowersMatch.fill(matchedTower)
                histoTowersReso.fill(reference=genParticle, target=matchedTower)

                ttCluster = clAlgo.buildTriggerTowerCluster(triggerTowers, matchedTower, debug)
                histoTowersResoCl.fill(reference=genParticle, target=ttCluster)

                # clustersInCone = sumClustersInCone(trigger3DClusters, allmatches[idx])
                # print ('----- in cone sum:')
                # print (clustersInCone)
                # histoResoCone.fill(reference=genParticle, target=clustersInCone.iloc[0])

                if debug >= 4:
                    print(f'--- Dump match for algo {algoname} ---------------')
                    print(f'GEN particle: idx: {idx}')
                    print(genParticle)
                    print('Matched Trigger Tower:')
                    print(matchedTower)
            elif debug >= 0:
                print(f'==== Warning no match found for algo {algoname}, idx {idx} ======================')
                if debug >= 2:
                    print(genParticle)



class ClusterTCGenMatchPlotter(BasePlotter):
    def __init__(self,  # ResoHistoClass,
                 data_set, gen_set,
                 data_selections=[selections.Selection('all')],
                 gen_selections=[selections.Selection('all')]):
        # self.ResoHistoClass = ResoHistoClass
        self.h_tcmatching = {}
        super(ClusterTCGenMatchPlotter, self).__init__(
            data_set,
            data_selections,
            gen_set,
            selections.multiply_selections(
                gen_selections,
                [selections.Selection('', '', 'gen > 0')]))

    def plotObjectMatch(self,
                        genParticles,
                        objects,
                        tcs,
                        h_tc_matched,
                        algoname,
                        debug):
        # fill histo with all selected GEN particles before any match

        best_match_indexes = {}
        if not objects.empty:
            best_match_indexes, allmatches = utils.match_etaphi(genParticles[['exeta', 'exphi']],
                                                                objects[['eta', 'phi']],
                                                                objects['pt'],
                                                                deltaR=0.1)
        if tcs.empty:
            return

        for idx, genParticle in genParticles.iterrows():
            if idx in best_match_indexes.keys():
                obj_matched = objects.loc[[best_match_indexes[idx]]].iloc[0]
                sel_tcs = tcs[(tcs.z * obj_matched.eta > 0)]
                if sel_tcs.empty:
                    continue

                sel_tcs.loc[sel_tcs.index, 'delta_eta'] = sel_tcs.eta - obj_matched.eta
                sel_tcs.loc[sel_tcs.index, 'delta_phi'] = sel_tcs.apply(lambda tc: ROOT.TVector2.Phi_mpi_pi(tc.phi - obj_matched.phi),
                                                                        axis=1)
                clAlgo.compute_tcs_to_cluster_deltaro(cluster=obj_matched,
                                                      tcs=sel_tcs)

                h_tc_matched.fill(sel_tcs, obj_matched)

                if debug >= 4:
                    print(f'--- Dump match for algo {algoname} ---------------')
                    print(f'GEN particle: idx: {idx}')
                    print(genParticle)

    def book_histos(self):
        self.gen_set.activate()
        self.data_set.activate()
        for tp_sel in self.data_selections:
            for gen_sel in self.gen_selections:
                histo_name = f'{self.data_set.name}_{tp_sel.name}_{gen_sel.name}'
                self.h_tcmatching[histo_name] = TCClusterMatchHistos(histo_name)

    def fill_histos(self, debug=0):
        for tp_sel in self.data_selections:
            objects = self.data_set.query(tp_sel)
            for gen_sel in self.gen_selections:
                genReference = self.gen_set.query(gen_sel)
                histo_name = f'{self.data_set.name}_{tp_sel.name}_{gen_sel.name}'

                h_tc_match = self.h_tcmatching[histo_name]
                # h_resoset = self.h_resoset[histo_name]
                # h_genseleff = self.h_effset[histo_name]
                # print 'TPsel: {}, GENsel: {}'.format(tp_sel.name, gen_sel.name)
                self.plotObjectMatch(genReference,
                                     objects,
                                     self.data_set.tcs.df,
                                     h_tc_match,
                                     self.data_set.name,
                                     debug)


class IsoTuplePlotter(BasePlotter):
    def __init__(self,
                 data_set, gen_set,
                 data_selections=[selections.Selection('all')],
                 gen_selections=[selections.Selection('all')]):
        self.h_resoset = {}
        super(IsoTuplePlotter, self).__init__(
            data_set,
            data_selections,
            gen_set,
            selections.multiply_selections(
                gen_selections,
                [selections.Selection('', '', 'gen > 0')]))

        # print self
        # print gen_selections

    def plotObjectMatch(self,
                        genParticles,
                        objects,
                        h_gen,
                        h_gen_matched,
                        h_object_matched,
                        h_reso,
                        algoname,
                        debug):
        # fill histo with all selected GEN particles before any match
        if h_gen:
            h_gen.fill(genParticles)

        best_match_indexes = {}
        if not objects.empty:
            best_match_indexes, allmatches = utils.match_etaphi(genParticles[['exeta', 'exphi']],
                                                                objects[['eta', 'phi']],
                                                                objects['pt'],
                                                                deltaR=0.1)
        # print best_match_indexes

        for idx, genParticle in genParticles.iterrows():
            if idx in best_match_indexes.keys():
                # print ('-----------------------')
                # print (genParticle)
                obj_matched = objects.loc[[best_match_indexes[idx]]]
                # h_object_matched.fill(obj_matched)
                h_reso.fill(reference=genParticle, target=obj_matched)

                if h_gen_matched is not None:
                    h_gen_matched.fill(genParticles.loc[[idx]])

        for idx, obj in objects.loc[~objects.index.isin(best_match_indexes.values())].iterrows():
            h_reso.fill(reference=None, target=obj)

    def book_histos(self):
        self.gen_set.activate()
        self.data_set.activate()
        for tp_sel in self.data_selections:
            for gen_sel in self.gen_selections:
                histo_name = f'{self.data_set.name}_{tp_sel.name}_{gen_sel.name}'
                self.h_resoset[histo_name] = IsoTuples(histo_name)

    def fill_histos(self, debug=0):
        pass

    def fill_histos_event(self, idx, debug=0):
        for tp_sel in self.data_selections:
            objects = self.data_set.query_event(tp_sel, idx)
            for gen_sel in self.gen_selections:
                genReference = self.gen_set.query_event(gen_sel, idx)
                histo_name = f'{self.data_set.name}_{tp_sel.name}_{gen_sel.name}'
                # print (histo_name)
                # print (f'# data: {objects.shape[0]}')
                # print (f'# gen: {genReference.shape[0]}')
                # print (genReference)
                h_resoset = self.h_resoset[histo_name]
                # print ('TPsel: {}, GENsel: {}'.format(tp_sel.name, gen_sel.name))
                self.plotObjectMatch(genReference,
                                     objects,
                                     None,
                                     None,
                                     None,
                                     h_resoset,
                                     self.data_set.name,
                                     debug)


class QuantizationPlotter(GenericDataFramePlotter):
# class QuantizationPlotter(GenericDataFrameLazyPlotter):
    def __init__(self, data_set, data_selections, features):
        self.features = features
        super(QuantizationPlotter, self).__init__(QuantizationHistos, data_set, data_selections)

    def book_histos(self):
        self.data_set.activate()
        data_name = self.data_set.name
        for selection in self.data_selections:
            self.h_set[selection.name] = self.HistoClass(
                name=f'{data_name}_{selection.name}_nomatch',
                features=self.features)


class CompTuplesPlotter(GenericDataFramePlotter):
    def __init__(self, obj_set, obj_selections=[selections.Selection('all')]):
        super(CompTuplesPlotter, self).__init__(CompTuples, obj_set, obj_selections)


class CompCatTuplePlotter(GenericGenMatchPlotter):
    def __init__(self, data_set, gen_set,
                 data_selections=[selections.Selection('all')],
                 gen_selections=[selections.Selection('all')]):
        super(CompCatTuplePlotter, self).__init__(EGHistos, CompCatTuples,
                                                data_set, gen_set,
                                                data_selections, gen_selections, drcut=0.2)


class HGCIdTuplesPlotter(GenericDataFramePlotter):
    def __init__(self, obj_set, obj_selections=[selections.Selection('all')]):
        super(HGCIdTuplesPlotter, self).__init__(HGCIdTuples, obj_set, obj_selections)



class HGCIdMatchTuplesPlotter(GenericGenMatchPlotter):
    def __init__(self, data_set, gen_set,
                 data_selections=[selections.Selection('all')],
                 gen_selections=[selections.Selection('all')]):
        super(HGCIdMatchTuplesPlotter, self).__init__(Cluster3DHistos, HGCIdMatchTuples,
                                                data_set, gen_set,
                                                data_selections, gen_selections, 
                                                drcut=0.1)


class DiObjMassPlotter(GenericDataFramePlotter):
    def __init__(self, obj_set, obj_selections=[selections.Selection('all')]):
        super(DiObjMassPlotter, self).__init__(DiObjMassHistos, obj_set, obj_selections)